##### Import libraries

In [1]:
# # Import Libraries
# import matplotlib
# matplotlib.use('agg')
# from google.colab import drive
# drive.mount('/content/gdrive')
# sys.path.append('/content/gdrive/My Drive/Colab Notebooks')
%load_ext line_profiler

import numpy as np
import pickle
import pandas as pd
import sys
import concurrent.futures
import time, random               # add some random sleep time
import scipy
import glob
import subprocess
import multiprocessing
import scipy.stats as stats
import time
import os
import math
import copy
import statsmodels.api as sm
import itertools
import re
import itertools
import six
import zipfile
import shutil
import h5py
from scipy.io import arff

from os import listdir

from spot import SPOT
from os.path import isfile, join
from statsutils import *
from boltons.statsutils import *
from datetime import datetime
from itertools import repeat
from ipywidgets import interact
from dateutil.parser import parse

from sklearn.preprocessing import normalize, StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import check_random_state, shuffle
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.svm import OneClassSVM as ocsvm
from sklearn import metrics, mixture,svm
from sklearn.neighbors import KNeighborsClassifier,LocalOutlierFactor
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler


from scipy import linalg
from scipy.special import gamma, factorial, digamma,betaln, gammaln
from scipy.stats import beta, multivariate_normal, wishart,invwishart,t, mode
from scipy.stats import genextreme as gev
import scipy.spatial as sp
import scipy.io
from scipy.io import arff

from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pylab as pylab
import matplotlib as mpl

sns.set(color_codes=True)
sns.set_style("whitegrid")
sns.set_context("paper", font_scale=2)
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
%matplotlib inline

Using TensorFlow backend.
/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/lekhag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/lekhag/opt/anaconda3/lib/python3.7

In [2]:
# SPOT Algorithm
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 12 10:08:16 2016

@author: Alban Siffer 
@company: Amossys
@license: GNU GPLv3
"""

from scipy.optimize import minimize
from math import log
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

# colors for plot
deep_saffron = '#FF9933'
air_force_blue = '#5D8AA8'


"""
================================= MAIN CLASS ==================================
"""

class SPOT:
    """
    This class allows to run SPOT algorithm on univariate dataset (upper-bound)
    
    Attributes
    ----------
    proba : float
        Detection level (risk), chosen by the user
        
    extreme_quantile : float
        current threshold (bound between normal and abnormal events)
        
    data : numpy.array
        stream
    
    init_data : numpy.array
        initial batch of observations (for the calibration/initialization step)
    
    init_threshold : float
        initial threshold computed during the calibration step
    
    peaks : numpy.array
        array of peaks (excesses above the initial threshold)
    
    n : int
        number of observed values
    
    Nt : int
        number of observed peaks
    """
    
    def __init__(self, q = 1e-4):
        """
        Constructor

	    Parameters
	    ----------
	    q
		    Detection level (risk)
	
	    Returns
	    ----------
    	SPOT object
        """
        self.proba = q
        self.extreme_quantile = None
        self.data = None
        self.init_data = None
        self.init_threshold = None
        self.peaks = None
        self.n = 0
        self.Nt = 0
        
    def __str__(self):
        s = ''
        s += 'Streaming Peaks-Over-Threshold Object\n'
        s += 'Detection level q = %s\n' % self.proba
        if self.data is not None:
            s += 'Data imported : Yes\n'
            s += '\t initialization  : %s values\n' % self.init_data.size
            s += '\t stream : %s values\n' % self.data.size
        else:
            s += 'Data imported : No\n'
            return s
            
        if self.n == 0:
            s += 'Algorithm initialized : No\n'
        else:
            s += 'Algorithm initialized : Yes\n'
            s += '\t initial threshold : %s\n' % self.init_threshold
            
            r = self.n-self.init_data.size
            if r > 0:
                s += 'Algorithm run : Yes\n'
                s += '\t number of observations : %s (%.2f %%)\n' % (r,100*r/self.n)
            else:
                s += '\t number of peaks  : %s\n' % self.Nt
                s += '\t extreme quantile : %s\n' % self.extreme_quantile
                s += 'Algorithm run : No\n'
        return s
    
    
    def fit(self,init_data,data):
        """
        Import data to SPOT object
        
        Parameters
	    ----------
	    init_data : list, numpy.array or pandas.Series
		    initial batch to calibrate the algorithm
            
        data : numpy.array
		    data for the run (list, np.array or pd.series)
	
        """
        if isinstance(data,list):
            self.data = np.array(data)
        elif isinstance(data,np.ndarray):
            self.data = data
        elif isinstance(data,pd.Series):
            self.data = data.values
        else:
            print('This data format (%s) is not supported' % type(data))
            return
            
        if isinstance(init_data,list):
            self.init_data = np.array(init_data)
        elif isinstance(init_data,np.ndarray):
            self.init_data = init_data
        elif isinstance(init_data,pd.Series):
            self.init_data = init_data.values
        elif isinstance(init_data,int):
            self.init_data = self.data[:init_data]
            self.data = self.data[init_data:]
        elif isinstance(init_data,float) & (init_data<1) & (init_data>0):
            r = int(init_data*data.size)
            self.init_data = self.data[:r]
            self.data = self.data[r:]
        else:
            print('The initial data cannot be set')
            return
        
    def add(self,data):
        """
        This function allows to append data to the already fitted data
        
        Parameters
	    ----------
	    data : list, numpy.array, pandas.Series
		    data to append
        """
        if isinstance(data,list):
            data = np.array(data)
        elif isinstance(data,np.ndarray):
            data = data
        elif isinstance(data,pd.Series):
            data = data.values
        else:
            print('This data format (%s) is not supported' % type(data))
            return
        
        self.data = np.append(self.data,data)
        return
    
    def initialize(self, verbose = True):
        """
        Run the calibration (initialization) step
        
        Parameters
	    ----------
	    verbose : bool
		    (default = True) If True, gives details about the batch initialization
        """
        n_init = self.init_data.size
        
        S = np.sort(self.init_data)     # we sort X to get the empirical quantile
        self.init_threshold = S[int(0.98*n_init)] # t is fixed for the whole algorithm

        # initial peaks
        self.peaks = self.init_data[self.init_data>self.init_threshold]-self.init_threshold 
        self.Nt = self.peaks.size
        self.n = n_init
        
        if verbose:
            print('Initial threshold : %s' % self.init_threshold)
            print('Number of peaks : %s' % self.Nt)
            print('Grimshaw maximum log-likelihood estimation ... ', end = '')
            
        g,s,l = self._grimshaw()
        self.extreme_quantile = self._quantile(g,s)
        
        if verbose:
            print('[done]')
            print('\t'+chr(0x03B3) + ' = ' + str(g))
            print('\t'+chr(0x03C3) + ' = ' + str(s))
            print('\tL = ' + str(l))
            print('Extreme quantile (probability = %s): %s' % (self.proba,self.extreme_quantile))
        
        return 
    
    
    
    
    def _rootsFinder(fun,jac,bounds,npoints,method):
        """
        Find possible roots of a scalar function
        
        Parameters
        ----------
        fun : function
		    scalar function 
        jac : function
            first order derivative of the function  
        bounds : tuple
            (min,max) interval for the roots search    
        npoints : int
            maximum number of roots to output      
        method : str
            'regular' : regular sample of the search interval, 'random' : uniform (distribution) sample of the search interval
        
        Returns
        ----------
        numpy.array
            possible roots of the function
        """
        if method == 'regular':
            step = (bounds[1]-bounds[0])/(npoints+1)
            X0 = np.arange(bounds[0]+step,bounds[1],step)
        elif method == 'random':
            X0 = np.random.uniform(bounds[0],bounds[1],npoints)
        
        def objFun(X,f,jac):
            g = 0
            j = np.zeros(X.shape)
            i = 0
            for x in X:
                fx = f(x)
                g = g+fx**2
                j[i] = 2*fx*jac(x)
                i = i+1
            return g,j
        
        opt = minimize(lambda X:objFun(X,fun,jac), X0, 
                       method='L-BFGS-B', 
                       jac=True, bounds=[bounds]*len(X0))
        
        X = opt.x
        np.round(X,decimals = 5)
        return np.unique(X)
    
    
    def _log_likelihood(Y,gamma,sigma):
        """
        Compute the log-likelihood for the Generalized Pareto Distribution (μ=0)
        
        Parameters
        ----------
        Y : numpy.array
		    observations
        gamma : float
            GPD index parameter
        sigma : float
            GPD scale parameter (>0)   

        Returns
        ----------
        float
            log-likelihood of the sample Y to be drawn from a GPD(γ,σ,μ=0)
        """
        n = Y.size
        if gamma != 0:
            tau = gamma/sigma
            L = -n * log(sigma) - ( 1 + (1/gamma) ) * ( np.log(1+tau*Y) ).sum()
        else:
            L = n * ( 1 + log(Y.mean()) )
        return L


    def _grimshaw(self,epsilon = 1e-8, n_points = 10):
        """
        Compute the GPD parameters estimation with the Grimshaw's trick
        
        Parameters
        ----------
        epsilon : float
		    numerical parameter to perform (default : 1e-8)
        n_points : int
            maximum number of candidates for maximum likelihood (default : 10)

        Returns
        ----------
        gamma_best,sigma_best,ll_best
            gamma estimates, sigma estimates and corresponding log-likelihood
        """
        def u(s):
            return 1 + np.log(s).mean()
            
        def v(s):
            return np.mean(1/s)
        
        def w(Y,t):
            s = 1+t*Y
            us = u(s)
            vs = v(s)
            return us*vs-1
        
        def jac_w(Y,t):
            s = 1+t*Y
            us = u(s)
            vs = v(s)
            jac_us = (1/t)*(1-vs)
            jac_vs = (1/t)*(-vs+np.mean(1/s**2))
            return us*jac_vs+vs*jac_us
            
    
        Ym = self.peaks.min()
        YM = self.peaks.max()
        Ymean = self.peaks.mean()
        
        
        a = -1/YM
        if abs(a)<2*epsilon:
            epsilon = abs(a)/n_points
        
        a = a + epsilon
        b = 2*(Ymean-Ym)/(Ymean*Ym)
        c = 2*(Ymean-Ym)/(Ym**2)
    
        # We look for possible roots
        left_zeros = SPOT._rootsFinder(lambda t: w(self.peaks,t),
                                 lambda t: jac_w(self.peaks,t),
                                 (a+epsilon,-epsilon),
                                 n_points,'regular')
        
        right_zeros = SPOT._rootsFinder(lambda t: w(self.peaks,t),
                                  lambda t: jac_w(self.peaks,t),
                                  (b,c),
                                  n_points,'regular')
    
        # all the possible roots
        zeros = np.concatenate((left_zeros,right_zeros))
        
        # 0 is always a solution so we initialize with it
        gamma_best = 0
        sigma_best = Ymean
        ll_best = SPOT._log_likelihood(self.peaks,gamma_best,sigma_best)
        
        # we look for better candidates
        for z in zeros:
            gamma = u(1+z*self.peaks)-1
            sigma = gamma/z
            ll = SPOT._log_likelihood(self.peaks,gamma,sigma)
            if ll>ll_best:
                gamma_best = gamma
                sigma_best = sigma
                ll_best = ll
    
        return gamma_best,sigma_best,ll_best

    

    def _quantile(self,gamma,sigma):
        """
        Compute the quantile at level 1-q
        
        Parameters
        ----------
        gamma : float
		    GPD parameter
        sigma : float
            GPD parameter

        Returns
        ----------
        float
            quantile at level 1-q for the GPD(γ,σ,μ=0)
        """
        r = self.n * self.proba / self.Nt
        if gamma != 0:
            return self.init_threshold + (sigma/gamma)*(pow(r,-gamma)-1)
        else:
            return self.init_threshold - sigma*log(r)

        
    def run(self, with_alarm = True):
        """
        Run SPOT on the stream
        
        Parameters
        ----------
        with_alarm : bool
		    (default = True) If False, SPOT will adapt the threshold assuming \
            there is no abnormal values


        Returns
        ----------
        dict
            keys : 'thresholds' and 'alarms'
            
            'thresholds' contains the extreme quantiles and 'alarms' contains \
            the indexes of the values which have triggered alarms
            
        """
        if (self.n>self.init_data.size):
            print('Warning : the algorithm seems to have already been run, you \
            should initialize before running again')
            return {}
        
        # list of the thresholds
        th = []
        alarm = []
        # Loop over the stream
        for i in tqdm.tqdm(range(self.data.size)):
    
            # If the observed value exceeds the current threshold (alarm case)
            if self.data[i]>self.extreme_quantile:
                # if we want to alarm, we put it in the alarm list
                if with_alarm:
                    alarm.append(i)
                # otherwise we add it in the peaks
                else:
                    self.peaks = np.append(self.peaks,self.data[i]-self.init_threshold)
                    self.Nt += 1
                    self.n += 1
                    # and we update the thresholds

                    g,s,l = self._grimshaw()
                    self.extreme_quantile = self._quantile(g,s)

            # case where the value exceeds the initial threshold but not the alarm ones
            elif self.data[i]>self.init_threshold:
                    # we add it in the peaks
                    self.peaks = np.append(self.peaks,self.data[i]-self.init_threshold)
                    self.Nt += 1
                    self.n += 1
                    # and we update the thresholds

                    g,s,l = self._grimshaw()
                    self.extreme_quantile = self._quantile(g,s)
            else:
                self.n += 1

                
            th.append(self.extreme_quantile) # thresholds record
        
        return {'thresholds' : th, 'alarms': alarm}
    

    def plot(self,run_results,with_alarm = True):
        """
        Plot the results of given by the run
        
        Parameters
        ----------
        run_results : dict
            results given by the 'run' method
        with_alarm : bool
		    (default = True) If True, alarms are plotted.


        Returns
        ----------
        list
            list of the plots
            
        """
        x = range(self.data.size)
        K = run_results.keys()
        
        ts_fig, = plt.plot(x,self.data,color="blue")
        fig = [ts_fig]
        ax=plt.gca()
        ax.set_facecolor('white')

        
        if 'thresholds' in K:
            th = run_results['thresholds']
            th_fig, = plt.plot(x,th,color=deep_saffron,lw=2,ls='dashed')
            fig.append(th_fig)
        
        if with_alarm and ('alarms' in K):
            alarm = run_results['alarms']
            al_fig = plt.scatter(alarm,self.data[alarm],color='red')
            fig.append(al_fig)
            
        plt.xlim((0,self.data.size))

        
        return fig
            









"""
============================ UPPER & LOWER BOUNDS =============================
"""




class biSPOT:
    """
    This class allows to run biSPOT algorithm on univariate dataset (upper and lower bounds)
    
    Attributes
    ----------
    proba : float
        Detection level (risk), chosen by the user
        
    extreme_quantile : float
        current threshold (bound between normal and abnormal events)
        
    data : numpy.array
        stream
    
    init_data : numpy.array
        initial batch of observations (for the calibration/initialization step)
    
    init_threshold : float
        initial threshold computed during the calibration step
    
    peaks : numpy.array
        array of peaks (excesses above the initial threshold)
    
    n : int
        number of observed values
    
    Nt : int
        number of observed peaks
    """
    def __init__(self, q = 1e-4):
        """
        Constructor

	    Parameters
	    ----------
	    q
		    Detection level (risk)
	
	    Returns
	    ----------
        biSPOT object
        """
        self.proba = q
        self.data = None
        self.init_data = None
        self.n = 0
        nonedict =  {'up':None,'down':None}
        
        self.extreme_quantile = dict.copy(nonedict)
        self.init_threshold = dict.copy(nonedict)
        self.peaks = dict.copy(nonedict)
        self.gamma = dict.copy(nonedict)
        self.sigma = dict.copy(nonedict)
        self.Nt = {'up':0,'down':0}
        
        
    def __str__(self):
        s = ''
        s += 'Streaming Peaks-Over-Threshold Object\n'
        s += 'Detection level q = %s\n' % self.proba
        if self.data is not None:
            s += 'Data imported : Yes\n'
            s += '\t initialization  : %s values\n' % self.init_data.size
            s += '\t stream : %s values\n' % self.data.size
        else:
            s += 'Data imported : No\n'
            return s
            
        if self.n == 0:
            s += 'Algorithm initialized : No\n'
        else:
            s += 'Algorithm initialized : Yes\n'
            s += '\t initial threshold : %s\n' % self.init_threshold
            
            r = self.n-self.init_data.size
            if r > 0:
                s += 'Algorithm run : Yes\n'
                s += '\t number of observations : %s (%.2f %%)\n' % (r,100*r/self.n)
                s += '\t triggered alarms : %s (%.2f %%)\n' % (len(self.alarm),100*len(self.alarm)/self.n)
            else:
                s += '\t number of peaks  : %s\n' % self.Nt
                s += '\t upper extreme quantile : %s\n' % self.extreme_quantile['up']
                s += '\t lower extreme quantile : %s\n' % self.extreme_quantile['down']
                s += 'Algorithm run : No\n'
        return s
    
    
    def fit(self,init_data,data):
        """
        Import data to biSPOT object
        
        Parameters
	    ----------
	    init_data : list, numpy.array or pandas.Series
		    initial batch to calibrate the algorithm ()
            
        data : numpy.array
		    data for the run (list, np.array or pd.series)
	
        """
        if isinstance(data,list):
            self.data = np.array(data)
        elif isinstance(data,np.ndarray):
            self.data = data
        elif isinstance(data,pd.Series):
            self.data = data.values
        else:
            print('This data format (%s) is not supported' % type(data))
            return
            
        if isinstance(init_data,list):
            self.init_data = np.array(init_data)
        elif isinstance(init_data,np.ndarray):
            self.init_data = init_data
        elif isinstance(init_data,pd.Series):
            self.init_data = init_data.values
        elif isinstance(init_data,int):
            self.init_data = self.data[:init_data]
            self.data = self.data[init_data:]
        elif isinstance(init_data,float) & (init_data<1) & (init_data>0):
            r = int(init_data*data.size)
            self.init_data = self.data[:r]
            self.data = self.data[r:]
        else:
            print('The initial data cannot be set')
            return
        
    def add(self,data):
        """
        This function allows to append data to the already fitted data
        
        Parameters
	    ----------
	    data : list, numpy.array, pandas.Series
		    data to append
        """
        if isinstance(data,list):
            data = np.array(data)
        elif isinstance(data,np.ndarray):
            data = data
        elif isinstance(data,pd.Series):
            data = data.values
        else:
            print('This data format (%s) is not supported' % type(data))
            return
        
        self.data = np.append(self.data,data)
        return

    def initialize(self, verbose = True):
        """
        Run the calibration (initialization) step
        
        Parameters
	    ----------
	    verbose : bool
		    (default = True) If True, gives details about the batch initialization
        """
        n_init = self.init_data.size
        
        S = np.sort(self.init_data)     # we sort X to get the empirical quantile
        self.init_threshold['up'] = S[int(0.98*n_init)] # t is fixed for the whole algorithm
        self.init_threshold['down'] = S[int(0.02*n_init)] # t is fixed for the whole algorithm

        # initial peaks
        self.peaks['up'] = self.init_data[self.init_data>self.init_threshold['up']]-self.init_threshold['up']
        self.peaks['down'] = -(self.init_data[self.init_data<self.init_threshold['down']]-self.init_threshold['down'])
        self.Nt['up'] = self.peaks['up'].size
        self.Nt['down'] = self.peaks['down'].size
        self.n = n_init
        
        if verbose:
            print('Initial threshold : %s' % self.init_threshold)
            print('Number of peaks : %s' % self.Nt)
            print('Grimshaw maximum log-likelihood estimation ... ', end = '')
            
        l = {'up':None,'down':None}
        for side in ['up','down']:
            g,s,l[side] = self._grimshaw(side)
            self.extreme_quantile[side] = self._quantile(side,g,s)
            self.gamma[side] = g
            self.sigma[side] = s
        
        ltab = 20
        form = ('\t'+'%20s' + '%20.2f' + '%20.2f')
        if verbose:
            print('[done]')
            print('\t' + 'Parameters'.rjust(ltab) + 'Upper'.rjust(ltab) + 'Lower'.rjust(ltab))
            print('\t' + '-'*ltab*3)
            print(form % (chr(0x03B3),self.gamma['up'],self.gamma['down']))
            print(form % (chr(0x03C3),self.sigma['up'],self.sigma['down']))
            print(form % ('likelihood',l['up'],l['down']))
            print(form % ('Extreme quantile',self.extreme_quantile['up'],self.extreme_quantile['down']))
            print('\t' + '-'*ltab*3)
        return 
    
    
    
    
    def _rootsFinder(fun,jac,bounds,npoints,method):
        """
        Find possible roots of a scalar function
        
        Parameters
        ----------
        fun : function
		    scalar function 
        jac : function
            first order derivative of the function  
        bounds : tuple
            (min,max) interval for the roots search    
        npoints : int
            maximum number of roots to output      
        method : str
            'regular' : regular sample of the search interval, 'random' : uniform (distribution) sample of the search interval
        
        Returns
        ----------
        numpy.array
            possible roots of the function
        """
        if method == 'regular':
            step = (bounds[1]-bounds[0])/(npoints+1)
            X0 = np.arange(bounds[0]+step,bounds[1],step)
        elif method == 'random':
            X0 = np.random.uniform(bounds[0],bounds[1],npoints)
        
        def objFun(X,f,jac):
            g = 0
            j = np.zeros(X.shape)
            i = 0
            for x in X:
                fx = f(x)
                g = g+fx**2
                j[i] = 2*fx*jac(x)
                i = i+1
            return g,j
        opt = minimize(lambda X:objFun(X,fun,jac), X0, 
                       method='L-BFGS-B', 
                       jac=True, bounds=[bounds]*len(X0))
        
        X = opt.x
        np.round(X,decimals = 5)
        return np.unique(X)
    
    
    def _log_likelihood(Y,gamma,sigma):
        """
        Compute the log-likelihood for the Generalized Pareto Distribution (μ=0)
        
        Parameters
        ----------
        Y : numpy.array
		    observations
        gamma : float
            GPD index parameter
        sigma : float
            GPD scale parameter (>0)   

        Returns
        ----------
        float
            log-likelihood of the sample Y to be drawn from a GPD(γ,σ,μ=0)
        """
        n = Y.size
        if gamma != 0:
            tau = gamma/sigma
            L = -n * log(sigma) - ( 1 + (1/gamma) ) * ( np.log(1+tau*Y) ).sum()
        else:
            L = n * ( 1 + log(Y.mean()) )
        return L


    def _grimshaw(self,side,epsilon = 1e-8, n_points = 10):
        """
        Compute the GPD parameters estimation with the Grimshaw's trick
        
        Parameters
        ----------
        epsilon : float
		    numerical parameter to perform (default : 1e-8)
        n_points : int
            maximum number of candidates for maximum likelihood (default : 10)

        Returns
        ----------
        gamma_best,sigma_best,ll_best
            gamma estimates, sigma estimates and corresponding log-likelihood
        """
        def u(s):
            return 1 + np.log(s).mean()
            
        def v(s):
            return np.mean(1/s)
        
        def w(Y,t):
            s = 1+t*Y
            us = u(s)
            vs = v(s)
            return us*vs-1
        
        def jac_w(Y,t):
            s = 1+t*Y
            us = u(s)
            vs = v(s)
            jac_us = (1/t)*(1-vs)
            jac_vs = (1/t)*(-vs+np.mean(1/s**2))
            return us*jac_vs+vs*jac_us
            
    
        Ym = self.peaks[side].min()
        YM = self.peaks[side].max()
        Ymean = self.peaks[side].mean()
        
        
        a = -1/YM
        if abs(a)<2*epsilon:
            epsilon = abs(a)/n_points
        
        a = a + epsilon
        b = 2*(Ymean-Ym)/(Ymean*Ym)
        c = 2*(Ymean-Ym)/(Ym**2)
    
        # We look for possible roots
        left_zeros = biSPOT._rootsFinder(lambda t: w(self.peaks[side],t),
                                 lambda t: jac_w(self.peaks[side],t),
                                 (a+epsilon,-epsilon),
                                 n_points,'regular')
        
        right_zeros = biSPOT._rootsFinder(lambda t: w(self.peaks[side],t),
                                  lambda t: jac_w(self.peaks[side],t),
                                  (b,c),
                                  n_points,'regular')
    
        # all the possible roots
        zeros = np.concatenate((left_zeros,right_zeros))
        
        # 0 is always a solution so we initialize with it
        gamma_best = 0
        sigma_best = Ymean
        ll_best = biSPOT._log_likelihood(self.peaks[side],gamma_best,sigma_best)
        
        # we look for better candidates
        for z in zeros:
            gamma = u(1+z*self.peaks[side])-1
            sigma = gamma/z
            ll = biSPOT._log_likelihood(self.peaks[side],gamma,sigma)
            if ll>ll_best:
                gamma_best = gamma
                sigma_best = sigma
                ll_best = ll
    
        return gamma_best,sigma_best,ll_best

    

    def _quantile(self,side,gamma,sigma):
        """
        Compute the quantile at level 1-q for a given side
        
        Parameters
        ----------
        side : str
            'up' or 'down'
        gamma : float
		    GPD parameter
        sigma : float
            GPD parameter

        Returns
        ----------
        float
            quantile at level 1-q for the GPD(γ,σ,μ=0)
        """
        if side == 'up':
            r = self.n * self.proba / self.Nt[side]
            if gamma != 0:
                return self.init_threshold['up'] + (sigma/gamma)*(pow(r,-gamma)-1)
            else:
                return self.init_threshold['up'] - sigma*log(r)
        elif side == 'down':
            r = self.n * self.proba / self.Nt[side]
            if gamma != 0:
                return self.init_threshold['down'] - (sigma/gamma)*(pow(r,-gamma)-1)
            else:
                return self.init_threshold['down'] + sigma*log(r)
        else:
            print('error : the side is not right')

        
    def run(self, with_alarm = True):
        """
        Run biSPOT on the stream
        
        Parameters
        ----------
        with_alarm : bool
		    (default = True) If False, SPOT will adapt the threshold assuming \
            there is no abnormal values


        Returns
        ----------
        dict
            keys : 'upper_thresholds', 'lower_thresholds' and 'alarms'
            
            '***-thresholds' contains the extreme quantiles and 'alarms' contains \
            the indexes of the values which have triggered alarms
            
        """
        if (self.n>self.init_data.size):
            print('Warning : the algorithm seems to have already been run, you \
            should initialize before running again')
            return {}
        
        # list of the thresholds
        thup = []
        thdown = []
        alarm = []
        # Loop over the stream
        for i in tqdm.tqdm(range(self.data.size)):
    
            # If the observed value exceeds the current threshold (alarm case)
            if self.data[i]>self.extreme_quantile['up'] :
                # if we want to alarm, we put it in the alarm list
                if with_alarm:
                    alarm.append(i)
                # otherwise we add it in the peaks
                else:
                    self.peaks['up'] = np.append(self.peaks['up'],self.data[i]-self.init_threshold['up'])
                    self.Nt['up'] += 1
                    self.n += 1
                    # and we update the thresholds

                    g,s,l = self._grimshaw('up')
                    self.extreme_quantile['up'] = self._quantile('up',g,s)

            # case where the value exceeds the initial threshold but not the alarm ones
            elif self.data[i]>self.init_threshold['up']:
                    # we add it in the peaks
                    self.peaks['up'] = np.append(self.peaks['up'],self.data[i]-self.init_threshold['up'])
                    self.Nt['up'] += 1
                    self.n += 1
                    # and we update the thresholds

                    g,s,l = self._grimshaw('up')
                    self.extreme_quantile['up'] = self._quantile('up',g,s)
                    
            elif self.data[i]<self.extreme_quantile['down'] :
                # if we want to alarm, we put it in the alarm list
                if with_alarm:
                    alarm.append(i)
                # otherwise we add it in the peaks
                else:
                    self.peaks['down'] = np.append(self.peaks['down'],-(self.data[i]-self.init_threshold['down']))
                    self.Nt['down'] += 1
                    self.n += 1
                    # and we update the thresholds

                    g,s,l = self._grimshaw('down')
                    self.extreme_quantile['down'] = self._quantile('down',g,s)

            # case where the value exceeds the initial threshold but not the alarm ones
            elif self.data[i]<self.init_threshold['down']:
                    # we add it in the peaks
                    self.peaks['down'] = np.append(self.peaks['down'],-(self.data[i]-self.init_threshold['down']))
                    self.Nt['down'] += 1
                    self.n += 1
                    # and we update the thresholds

                    g,s,l = self._grimshaw('down')
                    self.extreme_quantile['down'] = self._quantile('down',g,s)
            else:
                self.n += 1

                
            thup.append(self.extreme_quantile['up']) # thresholds record
            thdown.append(self.extreme_quantile['down']) # thresholds record
        
        return {'upper_thresholds' : thup,'lower_thresholds' : thdown, 'alarms': alarm}
    
    def plot(self,run_results,with_alarm = True):
        """
        Plot the results of given by the run
        
        Parameters
        ----------
        run_results : dict
            results given by the 'run' method
        with_alarm : bool
		    (default = True) If True, alarms are plotted.


        Returns
        ----------
        list
            list of the plots
            
        """
        x = range(self.data.size)
        K = run_results.keys()
        
        ts_fig, = plt.plot(x,self.data,color="blue")
        fig = [ts_fig]
        ax=plt.gca()
        ax.set_facecolor('white')

        if 'upper_thresholds' in K:
            thup = run_results['upper_thresholds']
            uth_fig, = plt.plot(x,thup,color=deep_saffron,lw=2,ls='dashed')
            fig.append(uth_fig)
            
        if 'lower_thresholds' in K:
            thdown = run_results['lower_thresholds']
            lth_fig, = plt.plot(x,thdown,color=deep_saffron,lw=2,ls='dashed')
            fig.append(lth_fig)
        
        if with_alarm and ('alarms' in K):
            alarm = run_results['alarms']
            al_fig = plt.scatter(alarm,self.data[alarm],color='red')
            fig.append(al_fig)
            
        plt.xlim((0,self.data.size))

        
        return fig








"""
================================= WITH DRIFT ==================================
"""

def backMean(X,d):
    M = []
    w = X[:d].sum()
    M.append(w/d)
    for i in range(d,len(X)):
        w = w - X[i-d] + X[i]
        M.append(w/d)
    return np.array(M)



class dSPOT:
    """
    This class allows to run DSPOT algorithm on univariate dataset (upper-bound)
    
    Attributes
    ----------
    proba : float
        Detection level (risk), chosen by the user
        
    depth : int
        Number of observations to compute the moving average
        
    extreme_quantile : float
        current threshold (bound between normal and abnormal events)
        
    data : numpy.array
        stream
    
    init_data : numpy.array
        initial batch of observations (for the calibration/initialization step)
    
    init_threshold : float
        initial threshold computed during the calibration step
    
    peaks : numpy.array
        array of peaks (excesses above the initial threshold)
    
    n : int
        number of observed values
    
    Nt : int
        number of observed peaks
    """
    def __init__(self, q, depth):
        self.proba = q
        self.extreme_quantile = None
        self.data = None
        self.init_data = None
        self.init_threshold = None
        self.peaks = None
        self.n = 0
        self.Nt = 0
        self.depth = depth
        
    def __str__(self):
        s = ''
        s += 'Streaming Peaks-Over-Threshold Object\n'
        s += 'Detection level q = %s\n' % self.proba
        if self.data is not None:
            s += 'Data imported : Yes\n'
            s += '\t initialization  : %s values\n' % self.init_data.size
            s += '\t stream : %s values\n' % self.data.size
        else:
            s += 'Data imported : No\n'
            return s
            
        if self.n == 0:
            s += 'Algorithm initialized : No\n'
        else:
            s += 'Algorithm initialized : Yes\n'
            s += '\t initial threshold : %s\n' % self.init_threshold
            
            r = self.n-self.init_data.size
            if r > 0:
                s += 'Algorithm run : Yes\n'
                s += '\t number of observations : %s (%.2f %%)\n' % (r,100*r/self.n)
                s += '\t triggered alarms : %s (%.2f %%)\n' % (len(self.alarm),100*len(self.alarm)/self.n)
            else:
                s += '\t number of peaks  : %s\n' % self.Nt
                s += '\t extreme quantile : %s\n' % self.extreme_quantile
                s += 'Algorithm run : No\n'
        return s
    
    
    def fit(self,init_data,data):
        """
        Import data to DSPOT object
        
        Parameters
	    ----------
	    init_data : list, numpy.array or pandas.Series
		    initial batch to calibrate the algorithm
            
        data : numpy.array
		    data for the run (list, np.array or pd.series)
	
        """
        if isinstance(data,list):
            self.data = np.array(data)
        elif isinstance(data,np.ndarray):
            self.data = data
        elif isinstance(data,pd.Series):
            self.data = data.values
        else:
            print('This data format (%s) is not supported' % type(data))
            return
            
        if isinstance(init_data,list):
            self.init_data = np.array(init_data)
        elif isinstance(init_data,np.ndarray):
            self.init_data = init_data
        elif isinstance(init_data,pd.Series):
            self.init_data = init_data.values
        elif isinstance(init_data,int):
            self.init_data = self.data[:init_data]
            self.data = self.data[init_data:]
        elif isinstance(init_data,float) & (init_data<1) & (init_data>0):
            r = int(init_data*data.size)
            self.init_data = self.data[:r]
            self.data = self.data[r:]
        else:
            print('The initial data cannot be set')
            return
        
    def add(self,data):
        """
        This function allows to append data to the already fitted data
        
        Parameters
	    ----------
	    data : list, numpy.array, pandas.Series
		    data to append
        """
        if isinstance(data,list):
            data = np.array(data)
        elif isinstance(data,np.ndarray):
            data = data
        elif isinstance(data,pd.Series):
            data = data.values
        else:
            print('This data format (%s) is not supported' % type(data))
            return
        
        self.data = np.append(self.data,data)
        return
    
    def initialize(self, verbose = True):
        """
        Run the calibration (initialization) step
        
        Parameters
	    ----------
	    verbose : bool
		    (default = True) If True, gives details about the batch initialization
        """
        n_init = self.init_data.size - self.depth
        
        M = backMean(self.init_data,self.depth)
        T = self.init_data[self.depth:]-M[:-1] # new variable
        
        S = np.sort(T)     # we sort X to get the empirical quantile
        self.init_threshold = S[int(0.98*n_init)] # t is fixed for the whole algorithm

        # initial peaks
        self.peaks = T[T>self.init_threshold]-self.init_threshold 
        self.Nt = self.peaks.size
        self.n = n_init
        
        if verbose:
            print('Initial threshold : %s' % self.init_threshold)
            print('Number of peaks : %s' % self.Nt)
            print('Grimshaw maximum log-likelihood estimation ... ', end = '')
            
        g,s,l = self._grimshaw()
        self.extreme_quantile = self._quantile(g,s)
        
        if verbose:
            print('[done]')
            print('\t'+chr(0x03B3) + ' = ' + str(g))
            print('\t'+chr(0x03C3) + ' = ' + str(s))
            print('\tL = ' + str(l))
            print('Extreme quantile (probability = %s): %s' % (self.proba,self.extreme_quantile))
        
        return
    
    
    
    
    def _rootsFinder(fun,jac,bounds,npoints,method):
        """
        Find possible roots of a scalar function
        
        Parameters
        ----------
        fun : function
		    scalar function 
        jac : function
            first order derivative of the function  
        bounds : tuple
            (min,max) interval for the roots search    
        npoints : int
            maximum number of roots to output      
        method : str
            'regular' : regular sample of the search interval, 'random' : uniform (distribution) sample of the search interval
        
        Returns
        ----------
        numpy.array
            possible roots of the function
        """
        if method == 'regular':
            step = (bounds[1]-bounds[0])/(npoints+1)
            X0 = np.arange(bounds[0]+step,bounds[1],step)
        elif method == 'random':
            X0 = np.random.uniform(bounds[0],bounds[1],npoints)
        
        def objFun(X,f,jac):
            g = 0
            j = np.zeros(X.shape)
            i = 0
            for x in X:
                fx = f(x)
                g = g+fx**2
                j[i] = 2*fx*jac(x)
                i = i+1
            return g,j
        
        opt = minimize(lambda X:objFun(X,fun,jac), X0, 
                       method='L-BFGS-B', 
                       jac=True, bounds=[bounds]*len(X0))
        
        X = opt.x
        np.round(X,decimals = 5)
        return np.unique(X)
    
    
    def _log_likelihood(Y,gamma,sigma):
        """
        Compute the log-likelihood for the Generalized Pareto Distribution (μ=0)
        
        Parameters
        ----------
        Y : numpy.array
		    observations
        gamma : float
            GPD index parameter
        sigma : float
            GPD scale parameter (>0)   

        Returns
        ----------
        float
            log-likelihood of the sample Y to be drawn from a GPD(γ,σ,μ=0)
        """
        n = Y.size
        if gamma != 0:
            tau = gamma/sigma
            L = -n * log(sigma) - ( 1 + (1/gamma) ) * ( np.log(1+tau*Y) ).sum()
        else:
            L = n * ( 1 + log(Y.mean()) )
        return L


    def _grimshaw(self,epsilon = 1e-8, n_points = 10):
        """
        Compute the GPD parameters estimation with the Grimshaw's trick
        
        Parameters
        ----------
        epsilon : float
		    numerical parameter to perform (default : 1e-8)
        n_points : int
            maximum number of candidates for maximum likelihood (default : 10)

        Returns
        ----------
        gamma_best,sigma_best,ll_best
            gamma estimates, sigma estimates and corresponding log-likelihood
        """
        def u(s):
            return 1 + np.log(s).mean()
            
        def v(s):
            return np.mean(1/s)
        
        def w(Y,t):
            s = 1+t*Y
            us = u(s)
            vs = v(s)
            return us*vs-1
        
        def jac_w(Y,t):
            s = 1+t*Y
            us = u(s)
            vs = v(s)
            jac_us = (1/t)*(1-vs)
            jac_vs = (1/t)*(-vs+np.mean(1/s**2))
            return us*jac_vs+vs*jac_us
            
    
        Ym = self.peaks.min()
        YM = self.peaks.max()
        Ymean = self.peaks.mean()
        
        
        a = -1/YM
        if abs(a)<2*epsilon:
            epsilon = abs(a)/n_points
        
        a = a + epsilon
        b = 2*(Ymean-Ym)/(Ymean*Ym)
        c = 2*(Ymean-Ym)/(Ym**2)
    
        # We look for possible roots
        left_zeros = SPOT._rootsFinder(lambda t: w(self.peaks,t),
                                 lambda t: jac_w(self.peaks,t),
                                 (a+epsilon,-epsilon),
                                 n_points,'regular')
        
        right_zeros = SPOT._rootsFinder(lambda t: w(self.peaks,t),
                                  lambda t: jac_w(self.peaks,t),
                                  (b,c),
                                  n_points,'regular')
    
        # all the possible roots
        zeros = np.concatenate((left_zeros,right_zeros))
        
        # 0 is always a solution so we initialize with it
        gamma_best = 0
        sigma_best = Ymean
        ll_best = SPOT._log_likelihood(self.peaks,gamma_best,sigma_best)
        
        # we look for better candidates
        for z in zeros:
            gamma = u(1+z*self.peaks)-1
            sigma = gamma/z
            ll = dSPOT._log_likelihood(self.peaks,gamma,sigma)
            if ll>ll_best:
                gamma_best = gamma
                sigma_best = sigma
                ll_best = ll
    
        return gamma_best,sigma_best,ll_best

    

    def _quantile(self,gamma,sigma):
        """
        Compute the quantile at level 1-q
        
        Parameters
        ----------
        gamma : float
		    GPD parameter
        sigma : float
            GPD parameter

        Returns
        ----------
        float
            quantile at level 1-q for the GPD(γ,σ,μ=0)
        """
        r = self.n * self.proba / self.Nt
        if gamma != 0:
            return self.init_threshold + (sigma/gamma)*(pow(r,-gamma)-1)
        else:
            return self.init_threshold - sigma*log(r)

        
    def run(self, with_alarm = True):
        """
        Run biSPOT on the stream
        
        Parameters
        ----------
        with_alarm : bool
		    (default = True) If False, SPOT will adapt the threshold assuming \
            there is no abnormal values


        Returns
        ----------
        dict
            keys : 'upper_thresholds', 'lower_thresholds' and 'alarms'
            
            '***-thresholds' contains the extreme quantiles and 'alarms' contains \
            the indexes of the values which have triggered alarms
            
        """
        if (self.n>self.init_data.size):
            print('Warning : the algorithm seems to have already been run, you \
            should initialize before running again')
            return {}
        
        # actual normal window
        W = self.init_data[-self.depth:]
        
        # list of the thresholds
        th = []
        alarm = []
        # Loop over the stream
        for i in tqdm.tqdm(range(self.data.size)):
            Mi = W.mean()
            # If the observed value exceeds the current threshold (alarm case)
            if (self.data[i]-Mi)>self.extreme_quantile:
                # if we want to alarm, we put it in the alarm list
                if with_alarm:
                    alarm.append(i)
                # otherwise we add it in the peaks
                else:
                    self.peaks = np.append(self.peaks,self.data[i]-Mi-self.init_threshold)
                    self.Nt += 1
                    self.n += 1
                    # and we update the thresholds

                    g,s,l = self._grimshaw()
                    self.extreme_quantile = self._quantile(g,s) #+ Mi
                    W = np.append(W[1:],self.data[i])

            # case where the value exceeds the initial threshold but not the alarm ones
            elif (self.data[i]-Mi)>self.init_threshold:
                    # we add it in the peaks
                    self.peaks = np.append(self.peaks,self.data[i]-Mi-self.init_threshold)
                    self.Nt += 1
                    self.n += 1
                    # and we update the thresholds

                    g,s,l = self._grimshaw()
                    self.extreme_quantile = self._quantile(g,s) #+ Mi
                    W = np.append(W[1:],self.data[i])
            else:
                self.n += 1
                W = np.append(W[1:],self.data[i])

                
            th.append(self.extreme_quantile+Mi) # thresholds record
        
        return {'thresholds' : th, 'alarms': alarm}
    

    def plot(self,run_results, with_alarm = True):
        """
        Plot the results given by the run
        
        Parameters
        ----------
        run_results : dict
            results given by the 'run' method
        with_alarm : bool
		    (default = True) If True, alarms are plotted.


        Returns
        ----------
        list
            list of the plots
            
        """
        x = range(self.data.size)
        K = run_results.keys()
        
        ts_fig, = plt.plot(x,self.data,color="blue")
        fig = [ts_fig]
        ax=plt.gca()
        ax.set_facecolor('white')
        
#        if 'upper_thresholds' in K:
#            thup = run_results['upper_thresholds']
#            uth_fig, = plt.plot(x,thup,color=deep_saffron,lw=2,ls='dashed')
#            fig.append(uth_fig)
#            
#        if 'lower_thresholds' in K:
#            thdown = run_results['lower_thresholds']
#            lth_fig, = plt.plot(x,thdown,color=deep_saffron,lw=2,ls='dashed')
#            fig.append(lth_fig)
        
        if 'thresholds' in K:
            th = run_results['thresholds']
            th_fig, = plt.plot(x,th,color=deep_saffron,lw=2,ls='dashed')
            fig.append(th_fig)
        
        if with_alarm and ('alarms' in K):
            alarm = run_results['alarms']
            if len(alarm)>0:
                plt.scatter(alarm,self.data[alarm],color='red')
            
        plt.xlim((0,self.data.size))

        
        return fig
            






"""
=========================== DRIFT & DOUBLE BOUNDS =============================
"""



class bidSPOT:
    """
    This class allows to run DSPOT algorithm on univariate dataset (upper and lower bounds)
    
    Attributes
    ----------
    proba : float
        Detection level (risk), chosen by the user
        
    depth : int
        Number of observations to compute the moving average
        
    extreme_quantile : float
        current threshold (bound between normal and abnormal events)
        
    data : numpy.array
        stream
    
    init_data : numpy.array
        initial batch of observations (for the calibration/initialization step)
    
    init_threshold : float
        initial threshold computed during the calibration step
    
    peaks : numpy.array
        array of peaks (excesses above the initial threshold)
    
    n : int
        number of observed values
    
    Nt : int
        number of observed peaks
    """
    def __init__(self, q = 1e-4, depth = 10):
        self.proba = q
        self.data = None
        self.init_data = None
        self.n = 0
        self.depth = depth
        
        nonedict =  {'up':None,'down':None}
        
        self.extreme_quantile = dict.copy(nonedict)
        self.init_threshold = dict.copy(nonedict)
        self.peaks = dict.copy(nonedict)
        self.gamma = dict.copy(nonedict)
        self.sigma = dict.copy(nonedict)
        self.Nt = {'up':0,'down':0}
        
        
    def __str__(self):
        s = ''
        s += 'Streaming Peaks-Over-Threshold Object\n'
        s += 'Detection level q = %s\n' % self.proba
        if self.data is not None:
            s += 'Data imported : Yes\n'
            s += '\t initialization  : %s values\n' % self.init_data.size
            s += '\t stream : %s values\n' % self.data.size
        else:
            s += 'Data imported : No\n'
            return s
            
        if self.n == 0:
            s += 'Algorithm initialized : No\n'
        else:
            s += 'Algorithm initialized : Yes\n'
            s += '\t initial threshold : %s\n' % self.init_threshold
            
            r = self.n-self.init_data.size
            if r > 0:
                s += 'Algorithm run : Yes\n'
                s += '\t number of observations : %s (%.2f %%)\n' % (r,100*r/self.n)
                s += '\t triggered alarms : %s (%.2f %%)\n' % (len(self.alarm),100*len(self.alarm)/self.n)
            else:
                s += '\t number of peaks  : %s\n' % self.Nt
                s += '\t upper extreme quantile : %s\n' % self.extreme_quantile['up']
                s += '\t lower extreme quantile : %s\n' % self.extreme_quantile['down']
                s += 'Algorithm run : No\n'
        return s
    
    
    def fit(self,init_data,data):
        """
        Import data to biDSPOT object
        
        Parameters
	    ----------
	    init_data : list, numpy.array or pandas.Series
		    initial batch to calibrate the algorithm
            
        data : numpy.array
		    data for the run (list, np.array or pd.series)
	
        """
        if isinstance(data,list):
            self.data = np.array(data)
        elif isinstance(data,np.ndarray):
            self.data = data
        elif isinstance(data,pd.Series):
            self.data = data.values
        else:
            print('This data format (%s) is not supported' % type(data))
            return
            
        if isinstance(init_data,list):
            self.init_data = np.array(init_data)
        elif isinstance(init_data,np.ndarray):
            self.init_data = init_data
        elif isinstance(init_data,pd.Series):
            self.init_data = init_data.values
        elif isinstance(init_data,int):
            self.init_data = self.data[:init_data]
            self.data = self.data[init_data:]
        elif isinstance(init_data,float) & (init_data<1) & (init_data>0):
            r = int(init_data*data.size)
            self.init_data = self.data[:r]
            self.data = self.data[r:]
        else:
            print('The initial data cannot be set')
            return
        
    def add(self,data):
        """
        This function allows to append data to the already fitted data
        
        Parameters
	    ----------
	    data : list, numpy.array, pandas.Series
		    data to append
        """
        if isinstance(data,list):
            data = np.array(data)
        elif isinstance(data,np.ndarray):
            data = data
        elif isinstance(data,pd.Series):
            data = data.values
        else:
            print('This data format (%s) is not supported' % type(data))
            return
        
        self.data = np.append(self.data,data)
        return
    
    def initialize(self, verbose = True):
        """
        Run the calibration (initialization) step
        
        Parameters
	    ----------
	    verbose : bool
		    (default = True) If True, gives details about the batch initialization
        """
        n_init = self.init_data.size - self.depth
        
        M = backMean(self.init_data,self.depth)
        T = self.init_data[self.depth:]-M[:-1] # new variable
        
        S = np.sort(T)     # we sort T to get the empirical quantile
        self.init_threshold['up'] = S[int(0.98*n_init)] # t is fixed for the whole algorithm
        self.init_threshold['down'] = S[int(0.02*n_init)] # t is fixed for the whole algorithm

        # initial peaks
        self.peaks['up'] = T[T>self.init_threshold['up']]-self.init_threshold['up']
        self.peaks['down'] = -( T[ T<self.init_threshold['down'] ] - self.init_threshold['down'] )
        self.Nt['up'] = self.peaks['up'].size
        self.Nt['down'] = self.peaks['down'].size
        self.n = n_init
        
        if verbose:
            print('Initial threshold : %s' % self.init_threshold)
            print('Number of peaks : %s' % self.Nt)
            print('Grimshaw maximum log-likelihood estimation ... ', end = '')
            
        l = {'up':None,'down':None}
        for side in ['up','down']:
            g,s,l[side] = self._grimshaw(side)
            self.extreme_quantile[side] = self._quantile(side,g,s)
            self.gamma[side] = g
            self.sigma[side] = s
        
        ltab = 20
        form = ('\t'+'%20s' + '%20.2f' + '%20.2f')
        if verbose:
            print('[done]')
            print('\t' + 'Parameters'.rjust(ltab) + 'Upper'.rjust(ltab) + 'Lower'.rjust(ltab))
            print('\t' + '-'*ltab*3)
            print(form % (chr(0x03B3),self.gamma['up'],self.gamma['down']))
            print(form % (chr(0x03C3),self.sigma['up'],self.sigma['down']))
            print(form % ('likelihood',l['up'],l['down']))
            print(form % ('Extreme quantile',self.extreme_quantile['up'],self.extreme_quantile['down']))
            print('\t' + '-'*ltab*3)
        return 
    
    
    
    
    def _rootsFinder(fun,jac,bounds,npoints,method):
        """
        Find possible roots of a scalar function
        
        Parameters
        ----------
        fun : function
		    scalar function 
        jac : function
            first order derivative of the function  
        bounds : tuple
            (min,max) interval for the roots search    
        npoints : int
            maximum number of roots to output      
        method : str
            'regular' : regular sample of the search interval, 'random' : uniform (distribution) sample of the search interval
        
        Returns
        ----------
        numpy.array
            possible roots of the function
        """
        if method == 'regular':
            step = (bounds[1]-bounds[0])/(npoints+1)
            X0 = np.arange(bounds[0]+step,bounds[1],step)
        elif method == 'random':
            X0 = np.random.uniform(bounds[0],bounds[1],npoints)
        
        def objFun(X,f,jac):
            g = 0
            j = np.zeros(X.shape)
            i = 0
            for x in X:
                fx = f(x)
                g = g+fx**2
                j[i] = 2*fx*jac(x)
                i = i+1
            return g,j
        
        opt = minimize(lambda X:objFun(X,fun,jac), X0, 
                       method='L-BFGS-B', 
                       jac=True, bounds=[bounds]*len(X0))
        
        X = opt.x
        np.round(X,decimals = 5)
        return np.unique(X)
    
    
    def _log_likelihood(Y,gamma,sigma):
        """
        Compute the log-likelihood for the Generalized Pareto Distribution (μ=0)
        
        Parameters
        ----------
        Y : numpy.array
		    observations
        gamma : float
            GPD index parameter
        sigma : float
            GPD scale parameter (>0)   

        Returns
        ----------
        float
            log-likelihood of the sample Y to be drawn from a GPD(γ,σ,μ=0)
        """
        n = Y.size
        if gamma != 0:
            tau = gamma/sigma
            L = -n * log(sigma) - ( 1 + (1/gamma) ) * ( np.log(1+tau*Y) ).sum()
        else:
            L = n * ( 1 + log(Y.mean()) )
        return L


    def _grimshaw(self,side,epsilon = 1e-8, n_points = 8):
        """
        Compute the GPD parameters estimation with the Grimshaw's trick
        
        Parameters
        ----------
        epsilon : float
		    numerical parameter to perform (default : 1e-8)
        n_points : int
            maximum number of candidates for maximum likelihood (default : 10)

        Returns
        ----------
        gamma_best,sigma_best,ll_best
            gamma estimates, sigma estimates and corresponding log-likelihood
        """
        def u(s):
            return 1 + np.log(s).mean()
            
        def v(s):
            return np.mean(1/s)
        
        def w(Y,t):
            s = 1+t*Y
            us = u(s)
            vs = v(s)
            return us*vs-1
        
        def jac_w(Y,t):
            s = 1+t*Y
            us = u(s)
            vs = v(s)
            jac_us = (1/t)*(1-vs)
            jac_vs = (1/t)*(-vs+np.mean(1/s**2))
            return us*jac_vs+vs*jac_us
            
    
        Ym = self.peaks[side].min()
        YM = self.peaks[side].max()
        Ymean = self.peaks[side].mean()
        
        
        a = -1/YM
        if abs(a)<2*epsilon:
            epsilon = abs(a)/n_points
        
        a = a + epsilon
        b = 2*(Ymean-Ym)/(Ymean*Ym)
        c = 2*(Ymean-Ym)/(Ym**2)
    
        # We look for possible roots
        left_zeros = bidSPOT._rootsFinder(lambda t: w(self.peaks[side],t),
                                 lambda t: jac_w(self.peaks[side],t),
                                 (a+epsilon,-epsilon),
                                 n_points,'regular')
        
        right_zeros = bidSPOT._rootsFinder(lambda t: w(self.peaks[side],t),
                                  lambda t: jac_w(self.peaks[side],t),
                                  (b,c),
                                  n_points,'regular')
    
        # all the possible roots
        zeros = np.concatenate((left_zeros,right_zeros))
        
        # 0 is always a solution so we initialize with it
        gamma_best = 0
        sigma_best = Ymean
        ll_best = bidSPOT._log_likelihood(self.peaks[side],gamma_best,sigma_best)
        
        # we look for better candidates
        for z in zeros:
            gamma = u(1+z*self.peaks[side])-1
            sigma = gamma/z
            ll = bidSPOT._log_likelihood(self.peaks[side],gamma,sigma)
            if ll>ll_best:
                gamma_best = gamma
                sigma_best = sigma
                ll_best = ll
    
        return gamma_best,sigma_best,ll_best

    

    def _quantile(self,side,gamma,sigma):
        """
        Compute the quantile at level 1-q for a given side
        
        Parameters
        ----------
        side : str
            'up' or 'down'
        gamma : float
		    GPD parameter
        sigma : float
            GPD parameter

        Returns
        ----------
        float
            quantile at level 1-q for the GPD(γ,σ,μ=0)
        """
        if side == 'up':
            r = self.n * self.proba / self.Nt[side]
            if gamma != 0:
                return self.init_threshold['up'] + (sigma/gamma)*(pow(r,-gamma)-1)
            else:
                return self.init_threshold['up'] - sigma*log(r)
        elif side == 'down':
            r = self.n * self.proba / self.Nt[side]
            if gamma != 0:
                return self.init_threshold['down'] - (sigma/gamma)*(pow(r,-gamma)-1)
            else:
                return self.init_threshold['down'] + sigma*log(r)
        else:
            print('error : the side is not right')

        
    def run(self, with_alarm = True, plot = True):
        """
        Run biDSPOT on the stream
        
        Parameters
        ----------
        with_alarm : bool
		    (default = True) If False, SPOT will adapt the threshold assuming \
            there is no abnormal values


        Returns
        ----------
        dict
            keys : 'upper_thresholds', 'lower_thresholds' and 'alarms'
            
            '***-thresholds' contains the extreme quantiles and 'alarms' contains \
            the indexes of the values which have triggered alarms
            
        """
        if (self.n>self.init_data.size):
            print('Warning : the algorithm seems to have already been run, you \
            should initialize before running again')
            return {}
        
        # actual normal window
        W = self.init_data[-self.depth:]
        
        # list of the thresholds
        thup = []
        thdown = []
        alarm = []
        # Loop over the stream
        for i in tqdm.tqdm(range(self.data.size)):
            Mi = W.mean()
            Ni = self.data[i]-Mi
            # If the observed value exceeds the current threshold (alarm case)
            if Ni>self.extreme_quantile['up'] :
                # if we want to alarm, we put it in the alarm list
                if with_alarm:
                    alarm.append(i)
                # otherwise we add it in the peaks
                else:
                    self.peaks['up'] = np.append(self.peaks['up'],Ni-self.init_threshold['up'])
                    self.Nt['up'] += 1
                    self.n += 1
                    # and we update the thresholds

                    g,s,l = self._grimshaw('up')
                    self.extreme_quantile['up'] = self._quantile('up',g,s)
                    W = np.append(W[1:],self.data[i])
                    
            # case where the value exceeds the initial threshold but not the alarm ones
            elif Ni>self.init_threshold['up']:
                    # we add it in the peaks
                    self.peaks['up'] = np.append(self.peaks['up'],Ni-self.init_threshold['up'])
                    self.Nt['up'] += 1
                    self.n += 1
                    # and we update the thresholds
                    g,s,l = self._grimshaw('up')
                    self.extreme_quantile['up'] = self._quantile('up',g,s)
                    W = np.append(W[1:],self.data[i])
                    
            elif Ni<self.extreme_quantile['down'] :
                # if we want to alarm, we put it in the alarm list
                if with_alarm:
                    alarm.append(i)
                # otherwise we add it in the peaks
                else:
                    self.peaks['down'] = np.append(self.peaks['down'],-(Ni-self.init_threshold['down']))
                    self.Nt['down'] += 1
                    self.n += 1
                    # and we update the thresholds

                    g,s,l = self._grimshaw('down')
                    self.extreme_quantile['down'] = self._quantile('down',g,s)
                    W = np.append(W[1:],self.data[i])
                    
            # case where the value exceeds the initial threshold but not the alarm ones
            elif Ni<self.init_threshold['down']:
                    # we add it in the peaks
                    self.peaks['down'] = np.append(self.peaks['down'],-(Ni-self.init_threshold['down']))
                    self.Nt['down'] += 1
                    self.n += 1
                    # and we update the thresholds

                    g,s,l = self._grimshaw('down')
                    self.extreme_quantile['down'] = self._quantile('down',g,s)
                    W = np.append(W[1:],self.data[i])
            else:
                self.n += 1
                W = np.append(W[1:],self.data[i])

                
            thup.append(self.extreme_quantile['up']+Mi) # upper thresholds record
            thdown.append(self.extreme_quantile['down']+Mi) # lower thresholds record
        
        return {'upper_thresholds' : thup,'lower_thresholds' : thdown, 'alarms': alarm}
    

    def plot(self,run_results, with_alarm = True):
        """
        Plot the results given by the run
        
        Parameters
        ----------
        run_results : dict
            results given by the 'run' method
        with_alarm : bool
		    (default = True) If True, alarms are plotted.


        Returns
        ----------
        list
            list of the plots
            
        """
        x = range(self.data.size)
        K = run_results.keys()
        
        ts_fig, = plt.plot(x,self.data,color="blue")
        fig = [ts_fig]
        ax=plt.gca()
        ax.set_facecolor('white')
        
        if 'upper_thresholds' in K:
            thup = run_results['upper_thresholds']
            uth_fig, = plt.plot(x,thup,color=deep_saffron,lw=2,ls='dashed')
            fig.append(uth_fig)
            
        if 'lower_thresholds' in K:
            thdown = run_results['lower_thresholds']
            lth_fig, = plt.plot(x,thdown,color=deep_saffron,lw=2,ls='dashed')
            fig.append(lth_fig)
        
        if with_alarm and ('alarms' in K):
            alarm = run_results['alarms']
            if len(alarm)>0:
                al_fig = plt.scatter(alarm,self.data[alarm],color='red')
                fig.append(al_fig)
            
        plt.xlim((0,self.data.size))

        
        return fig

In [3]:
# Global Figure Parameters
import matplotlib.pylab as pylab
plot_params = {'legend.fontsize': 60,
          'figure.figsize': (25*2, 15*2),
         'axes.labelsize': 60*2,
         'axes.titlesize':80*2,
         'xtick.labelsize':40*2,
         'ytick.labelsize':40*2}
pylab.rcParams.update(plot_params)
global fig_len
global fig_wid
global m_size

fig_len=8
fig_wid=8
m_size=100

In [4]:
def eval_perf(X,y):
    np.random.seed(4323)
    labels=y
    outliers_fraction= np.sum(y)/len(y)
    eval_metrics={}
    eval_preds={}
#     ros = RandomOverSampler(random_state=0)
#     X_resampled, y_resampled = ros.fit_resample(np.arange(0,len(X),1).reshape(-1,1), y)
#     train, test = train_test_split(X_resampled, test_size=0.2,random_state=200)
#     train=train[:,0]
    train=np.arange(0,len(X),1)
    if 1:
        eval_metrics['LOF']={}
        eval_metrics['LOF']['fpr']={}
        eval_metrics['LOF']['tpr']={}
        eval_metrics['LOF']['thresholds']={}
        eval_metrics['LOF']['fp']={}
        eval_metrics['LOF']['tp']={}
        eval_metrics['LOF']['fn']={}
        eval_metrics['LOF']['tn']={}
        eval_metrics['LOF']['recall']={}
        eval_metrics['LOF']['specificity']={}
        eval_metrics['LOF']['precision']={}
        eval_metrics['LOF']['accuracy']={}
        eval_metrics['LOF']['fmeasure']={}
        eval_metrics['LOF']['purity']={}
        eval_metrics['LOF']['auc']={}    

        eval_metrics['Kmeans--']={}
        eval_metrics['Kmeans--']['fpr']={}
        eval_metrics['Kmeans--']['tpr']={}
        eval_metrics['Kmeans--']['thresholds']={}
        eval_metrics['Kmeans--']['fp']={}
        eval_metrics['Kmeans--']['tp']={}
        eval_metrics['Kmeans--']['fn']={}
        eval_metrics['Kmeans--']['tn']={}
        eval_metrics['Kmeans--']['recall']={}
        eval_metrics['Kmeans--']['specificity']={}
        eval_metrics['Kmeans--']['precision']={}
        eval_metrics['Kmeans--']['accuracy']={}
        eval_metrics['Kmeans--']['fmeasure']={}
        eval_metrics['Kmeans--']['purity']={}
        eval_metrics['Kmeans--']['auc']={}    

        eval_metrics['knn']={}
        eval_metrics['knn']['fpr']={}
        eval_metrics['knn']['tpr']={}
        eval_metrics['knn']['thresholds']={}
        eval_metrics['knn']['fp']={}
        eval_metrics['knn']['tp']={}
        eval_metrics['knn']['fn']={}
        eval_metrics['knn']['tn']={}
        eval_metrics['knn']['recall']={}
        eval_metrics['knn']['specificity']={}
        eval_metrics['knn']['precision']={}
        eval_metrics['knn']['accuracy']={}
        eval_metrics['knn']['fmeasure']={}
        eval_metrics['knn']['purity']={}
        eval_metrics['knn']['auc']={}    

        eval_metrics['ocsvm']={}
        eval_metrics['ocsvm']['fpr']={}
        eval_metrics['ocsvm']['tpr']={}
        eval_metrics['ocsvm']['thresholds']={}
        eval_metrics['ocsvm']['fp']={}
        eval_metrics['ocsvm']['tp']={}
        eval_metrics['ocsvm']['fn']={}
        eval_metrics['ocsvm']['tn']={}
        eval_metrics['ocsvm']['recall']={}
        eval_metrics['ocsvm']['specificity']={}
        eval_metrics['ocsvm']['precision']={}
        eval_metrics['ocsvm']['accuracy']={}
        eval_metrics['ocsvm']['fmeasure']={}
        eval_metrics['ocsvm']['purity']={}
        eval_metrics['ocsvm']['auc']={}    


    # # try lof
    for nn in range(10,90,10):
        # fit the model for outlier detection (default)
        clf = LocalOutlierFactor(n_neighbors=nn, contamination=outliers_fraction)
        # use fit_predict to compute the predicted labels of the training samples
        # (when LOF is used for outlier detection, the estimator has no predict,
        # decision_function and score_samples methods).
        y_pred = clf.fit_predict(X)
        n_errors = (y_pred != y).sum()
        pred = clf.negative_outlier_factor_ 

        preds=np.zeros(len(np.array(pred)))
        preds[np.argsort(-np.array(pred))[0:int(sum(y))]]=1
        tn, fp, fn, tp =confusion_matrix(y,preds).ravel().astype(float)
        fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
        recall=tp/(tp+fn)
        specificity=tn/(tn+fp)
        precision=tp/(tp+fp)
        accuracy=(tp+tn)/(tp+fp+tn+fn)
        fmeasure=2*precision*recall/(precision + recall)
        purity=purity_score(y, preds)
        fpr, tpr, thresholds = metrics.roc_curve(y, pred)
        auc=metrics.auc(fpr, tpr)

        eval_preds['LOF']=preds
        eval_metrics['LOF']['fpr'][nn]=fpr
        eval_metrics['LOF']['tpr'][nn]=tpr
        eval_metrics['LOF']['thresholds'][nn]=thresholds
        eval_metrics['LOF']['fp'][nn]=fp
        eval_metrics['LOF']['tp'][nn]=tp
        eval_metrics['LOF']['fn'][nn]=fn
        eval_metrics['LOF']['tn'][nn]=tn
        eval_metrics['LOF']['recall'][nn]=recall
        eval_metrics['LOF']['specificity'][nn]=specificity
        eval_metrics['LOF']['precision'][nn]=precision
        eval_metrics['LOF']['accuracy'][nn]=accuracy
        eval_metrics['LOF']['fmeasure'][nn]=fmeasure
        eval_metrics['LOF']['purity'][nn]=purity
        eval_metrics['LOF']['auc'][nn]=auc

    # # Kmeans--
    ks = [1,3,5,7,9,11,21]
    for i in range(len(ks)):   
        if ks[i]<len(X):
            linds, C, c = kmeans__(X,ks[i],int(np.sum(labels)))
            preds = np.zeros([labels.shape[0],])
            preds[linds] = 1
            tn, fp, fn, tp =confusion_matrix(y,preds).ravel().astype(float)
            recall=tp/(tp+fn)
            specificity=tn/(tn+fp)
            precision=tp/(tp+fp)
            accuracy=(tp+tn)/(tp+fp+tn+fn)
            fmeasure=2*precision*recall/(precision + recall)
            fpr, tpr, thresholds = metrics.roc_curve(y, preds)
            purity=purity_score(y, preds)
            eval_preds['Kmeans-- '+str(ks[i])]=preds
            eval_metrics['Kmeans--']['fpr'][ks[i]]=fpr
            eval_metrics['Kmeans--']['tpr'][ks[i]]=tpr
            eval_metrics['Kmeans--']['thresholds'][ks[i]]=thresholds
            eval_metrics['Kmeans--']['fp'][ks[i]]=fp
            eval_metrics['Kmeans--']['tp'][ks[i]]=tp
            eval_metrics['Kmeans--']['fn'][ks[i]]=fn
            eval_metrics['Kmeans--']['tn'][ks[i]]=tn
            eval_metrics['Kmeans--']['recall'][ks[i]]=recall
            eval_metrics['Kmeans--']['specificity'][ks[i]]=specificity
            eval_metrics['Kmeans--']['precision'][ks[i]]=precision
            eval_metrics['Kmeans--']['accuracy'][ks[i]]=accuracy
            eval_metrics['Kmeans--']['fmeasure'][ks[i]]=fmeasure
            eval_metrics['Kmeans--']['purity'][ks[i]]=purity
            eval_metrics['Kmeans--']['auc'][ks[i]]=auc

    # # KNN
    ks = [1,3,5,7,9,11,21]
    for i in range(len(ks)):    
        #try knn
        knn = KNeighborsClassifier(n_neighbors=ks[i])
        knn.fit(X[train], y[train])
        preds=knn.predict(X)
        pred = preds
        preds=np.zeros(len(np.array(pred)))
        preds[np.argsort(-np.array(pred))[0:int(sum(y))]]=1
        tn, fp, fn, tp =confusion_matrix(y,preds).ravel().astype(float)
        recall=tp/(tp+fn)
        specificity=tn/(tn+fp)
        precision=tp/(tp+fp)
        accuracy=(tp+tn)/(tp+fp+tn+fn)
        fmeasure=2*precision*recall/(precision + recall)
        purity=purity_score(y, preds)
        fpr, tpr, thresholds = metrics.roc_curve(y, pred)
        auc=metrics.auc(fpr, tpr)
        eval_preds['knn '+str(ks[i])]=preds
        eval_metrics['knn']['fpr'][ks[i]]=fpr
        eval_metrics['knn']['tpr'][ks[i]]=tpr
        eval_metrics['knn']['thresholds'][ks[i]]=thresholds
        eval_metrics['knn']['fp'][ks[i]]=fp
        eval_metrics['knn']['tp'][ks[i]]=tp
        eval_metrics['knn']['fn'][ks[i]]=fn
        eval_metrics['knn']['tn'][ks[i]]=tn
        eval_metrics['knn']['recall'][ks[i]]=recall
        eval_metrics['knn']['specificity'][ks[i]]=specificity
        eval_metrics['knn']['precision'][ks[i]]=precision
        eval_metrics['knn']['accuracy'][ks[i]]=accuracy
        eval_metrics['knn']['fmeasure'][ks[i]]=fmeasure
        eval_metrics['knn']['purity'][ks[i]]=purity
        eval_metrics['knn']['auc'][ks[i]]=auc

#     # try ocsvm
    for gmma in np.arange(0.05,1,0.05):
        ##print(gmma)
#             oc = ocsvm(nu=nu,gamma=gmma)
        oc=svm.OneClassSVM(nu=outliers_fraction, kernel="rbf",gamma=gmma)
        oc.fit(X[train])
        p = oc.predict(X)
        preds = np.zeros(p.shape)
        preds[p == -1] = 1
        preds[p == 1] = 0
        tn, fp, fn, tp =confusion_matrix(y,preds).ravel().astype(float)
        recall=tp/(tp+fn)
        specificity=tn/(tn+fp)
        precision=tp/(tp+fp)
        accuracy=(tp+tn)/(tp+fp+tn+fn)
        fmeasure=2*precision*recall/(precision + recall)
        purity=purity_score(y, preds)
        fpr, tpr, thresholds = metrics.roc_curve(y, preds)
        auc=metrics.auc(fpr, tpr)

        eval_preds['ocsvm '+str(gmma)]=preds
        eval_metrics['ocsvm']['fpr'][gmma]=fpr
        eval_metrics['ocsvm']['tpr'][gmma]=tpr
        eval_metrics['ocsvm']['thresholds'][gmma]=thresholds
        eval_metrics['ocsvm']['fp'][gmma]=fp
        eval_metrics['ocsvm']['tp'][gmma]=tp
        eval_metrics['ocsvm']['fn'][gmma]=fn
        eval_metrics['ocsvm']['tn'][gmma]=tn
        eval_metrics['ocsvm']['recall'][gmma]=recall
        eval_metrics['ocsvm']['specificity'][gmma]=specificity
        eval_metrics['ocsvm']['precision'][gmma]=precision
        eval_metrics['ocsvm']['accuracy'][gmma]=accuracy
        eval_metrics['ocsvm']['fmeasure'][gmma]=fmeasure
        eval_metrics['ocsvm']['purity'][gmma]=purity
        eval_metrics['ocsvm']['auc'][gmma]=auc

    return eval_metrics,eval_preds

In [5]:
def plotClusters(thetas,z,samples):
    thetameans = []
    K = len(thetas)
    plt.figure(figsize=(fig_len,fig_wid))
    ax=plt.gca()
    ax.set_facecolor('white')
    ax.tick_params(labelsize=25)
    ax.set_facecolor('white')
    ax.grid(color='k', linestyle='-.', linewidth=0.3)

    for k in range(K):
        thetameans.append(thetas[k][0])
    thetameans = np.array(thetameans)
    for k in range(K):
        plt.scatter(samples[z == k,0],samples[z == k,1],marker='*',s=m_size)
    plt.legend([str(k) for k in range(K)])
    #plt.scatter(thetameans[:,0],thetameans[:,1],marker='x')
    for k in range(K):
        plt.text(thetameans[k,0],thetameans[k,1],str(k))
def multivariatet(mu,Sigma,N,M):
    '''
    Output:
    Produce M samples of d-dimensional multivariate t distribution
    Input:
    mu = mean (d dimensional numpy array or scalar)
    Sigma = scale matrix (dxd numpy array)
    N = degrees of freedom
    M = # of samples to produce
    '''
    d = len(Sigma)
    g = np.tile(np.random.gamma(N/2.,2./N,M),(d,1)).T
    Z = np.random.multivariate_normal(np.zeros(d),Sigma,M)
    return mu + Z/np.sqrt(g)
def normalinvwishartsample(params):
    '''
    Generate sample from a Normal Inverse Wishart distribution

    Inputs:
    params - Parameters for the NIW distribution 
        mu    - Mean parameter: n x 1 numpy array
        W     - Precision parameter: d x d numpy array
        kappa - Scalar parameter for normal distribution covariance matrix
        nu    - Scalar parameter for Wishart distribution

    Output:
    Sample - Sample mean vector, mu_s and Sample covariance matrix, W_s
    '''
    mu,W,kappa,nu = params
    # first sample W from a Inverse Wishart distribution
    W_s = invwishart(df=nu, scale=W).rvs()
    mu_s = np.random.multivariate_normal(mu.flatten(),W_s/kappa,1) 
    return np.transpose(mu_s),W_s
def normalinvwishartmarginal(X,params):
    '''
    Marginal likelihood of dataset X using a Normal Inverse Wishart prior

    Inputs:
    X      - Dataset matrix: n x d numpy array
    params - Parameters for the NIW distribution 
        mu    - Mean parameter: n x 1 numpy array
        W     - Precision parameter: d x d numpy array
        kappa - Scalar parameter for normal distribution covariance matrix
        nu    - Scalar parameter for Wishart distribution

    Output:
    Marginal likelihood of X - scalar
    '''
    mu,W,kappa,nu = params
    mu=X

    n = X.shape[0]
    d = X.shape[1]
    nu_n = nu + n
    kappa_n = kappa + n
    X_mean = np.mean(X,axis=0)
    X_mean = X_mean[:,np.newaxis]
    S = scatter(X)
    W_n = W + S + ((kappa*n)/(kappa+n))*np.dot(mu - X_mean,np.transpose(mu - X_mean))
    #(1/np.power(np.pi,n*d*0.5))*(gamma(nu_n*0.5)/gamma(nu*0.5))*(np.power(np.linalg.det(W),nu*0.5)/np.power(np.linalg.det(W_n),nu_n*0.5))*np.power(kappa/kappa_n,0.5*d)
    return (1/np.power(np.pi,n*d*0.5))*(gamma(nu_n*0.5)/gamma(nu*0.5))*(np.power(np.linalg.det(W)/np.linalg.det(W_n),nu*0.5)/np.power(np.linalg.det(W_n),(nu_n-nu)*0.5))*np.power(kappa/kappa_n,0.5*d)
def scatter(x):
    return np.dot(np.transpose(x - np.mean(x,0)),x - np.mean(x,0))
def plotAnomalies(I,samples):
    for k in range(2):
        plt.figure(figsize=(fig_len,fig_wid))
        ax=plt.gca()
        ax.set_facecolor('white')
        ax.tick_params(labelsize=20)
        ax.set_facecolor('white')
        ax.grid(color='k', linestyle='-.', linewidth=0.3)
        plt.scatter(samples[I == k,0],samples[I == k,1],marker='*',s=m_size)
def kmeans__(data,k,l,maxiters=100,eps=0.0001):

    # select k cluster centers
    C = data[np.random.permutation(range(data.shape[0]))[0:k],:]
    objVal = 0
    for jj in range(maxiters):
        # compute distance of each point to the clusters
        dMat = pdist2(data,C)
        d = np.min(dMat,axis=1).flatten()
        c = np.argmin(dMat,axis=1).flatten()
        # sort points by distance to their closest center
        inds = np.argsort(d)[::-1]
        linds = inds[0:l]
        cinds = inds[l+1:]
        # extract the non-outlier data objects
        ci = c[cinds]
        # recompute the means
        for kk in range(k):
            C[kk,:] = np.mean(data[np.where(ci == kk)[0],:],axis=0)
        # compute objective function
        objVal_ = objVal
        objVal = 0
        for kk in range(k):
            objVal += np.sum(pdist2(data[np.where(ci == kk)[0],:],C[kk,:]))
        if np.abs(objVal - objVal_) < eps:
            break
    # one final time
    dMat = pdist2(data,C)
    c = np.argmin(dMat,axis=1).flatten()
    return linds, C, c
def pdist2(X,C):
    if len(C.shape) == 1:
        C = C[:,np.newaxis]
    distMat = np.zeros([X.shape[0],C.shape[0]])
    for i in range(X.shape[0]):
        for j in range(C.shape[0]):
            distMat[i,j] = np.linalg.norm(X[i,:] - C[j,:])
    return distMat
def precAtK(true,predicted):
    # find number of anomalies
    k = np.sum(true)
#     print("k=",k)
    # find the score of the k^th predicted anomaly
    v = np.sort(predicted,axis=0)[::-1][k-1]
#     print("v=",v)
    # find all objects that are above the threshold
    inds = np.where(predicted >= v)[0]
#     print("inds=",inds)
#     print("np.sum(true[inds])=",np.sum(true[inds]))
#     print("len(inds)=",len(inds))
#     print("np.sum(true[inds])/len(inds)=",np.sum(true[inds])/len(inds))
    return float(np.sum(true[inds]))/float(len(inds))
def averageRank(true,predicted):
    inds = np.where(true == 1)[0]
    s = np.argsort(predicted)[::-1]
    v = []
    for ind in inds:
        v.append(float(np.where(s == ind)[0]+1))
    return np.mean(v)
def purity_score(y_true, y_pred):
    """Purity score

    To compute purity, each cluster is assigned to the class which is most frequent 
    in the cluster [1], and then the accuracy of this assignment is measured by counting 
    the number of correctly assigned documents and dividing by the number of documents.
    We suppose here that the ground truth labels are integers, the same with the predicted clusters i.e
    the clusters index.

    Args:
        y_true(np.ndarray): n*1 matrix Ground truth labels
        y_pred(np.ndarray): n*1 matrix Predicted clusters
    
    Returns:
        float: Purity score
    
    References:
        [1] https://nlp.stanford.edu/IR-book/html/htmledition/evaluation-of-clustering-1.html
    """
    # matrix which will hold the majority-voted labels
    y_voted_labels = np.zeros(y_true.shape)
    # Ordering labels
    ## Labels might be missing e.g with set like 0,2 where 1 is missing
    ## First find the unique labels, then map the labels to an ordered set
    ## 0,2 should become 0,1
    labels = np.unique(y_true)
    ordered_labels = np.arange(labels.shape[0])
    for k in range(labels.shape[0]):
        y_true[y_true==labels[k]] = ordered_labels[k]
    # Update unique labels
    labels = np.unique(y_true)
    # We set the number of bins to be n_classes+2 so that 
    # we count the actual occurence of classes between two consecutive bin
    # the bigger being excluded [bin_i, bin_i+1[
    bins = np.concatenate((labels, [np.max(labels)+1]), axis=0)

    for cluster in np.unique(y_pred):
        hist, _ = np.histogram(y_true[y_pred==cluster], bins=bins)
        # Find the most present label in the cluster
        winner = np.argmax(hist)
        y_voted_labels[y_pred==cluster] = winner
    
    return accuracy_score(y_true, y_voted_labels)


In [6]:
def _estimate_log_gaussian_prob(X, means, precisions_chol, covariance_type):
    """Estimate the log Gaussian probability.
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
    means : array-like, shape (n_components, n_features)
    precisions_chol : array-like
        Cholesky decompositions of the precision matrices.
        'full' : shape of (n_components, n_features, n_features)
        'tied' : shape of (n_features, n_features)
        'diag' : shape of (n_components, n_features)
        'spherical' : shape of (n_components,)
    covariance_type : {'full', 'tied', 'diag', 'spherical'}
    Returns
    -------
    log_prob : array, shape (n_samples, n_components)
    """
    n_samples, n_features = X.shape
    n_components, _ = means.shape
    # det(precision_chol) is half of det(precision)
    log_det = _compute_log_det_cholesky(
        precisions_chol, covariance_type, n_features)

    if covariance_type == 'full':
        log_prob = np.empty((n_samples, n_components))
        for k, (mu, prec_chol) in enumerate(zip(means, precisions_chol)):
            y = np.dot(X, prec_chol) - np.dot(mu, prec_chol)
            log_prob[:, k] = np.sum(np.square(y), axis=1)

    elif covariance_type == 'tied':
        log_prob = np.empty((n_samples, n_components))
        for k, mu in enumerate(means):
            y = np.dot(X, precisions_chol) - np.dot(mu, precisions_chol)
            log_prob[:, k] = np.sum(np.square(y), axis=1)

    elif covariance_type == 'diag':
        precisions = precisions_chol ** 2
        log_prob = (np.sum((means ** 2 * precisions), 1) -
                    2. * np.dot(X, (means * precisions).T) +
                    np.dot(X ** 2, precisions.T))

    elif covariance_type == 'spherical':
        precisions = precisions_chol ** 2
        log_prob = (np.sum(means ** 2, 1) * precisions -
                    2 * np.dot(X, means.T * precisions) +
                    np.outer(row_norms(X, squared=True), precisions))
    return -.5 * (n_features * np.log(2 * np.pi) + log_prob) + log_det
def _compute_precision_cholesky(covariances, covariance_type):
    """Compute the Cholesky decomposition of the precisions.
    Parameters
    ----------
    covariances : array-like
        The covariance matrix of the current components.
        The shape depends of the covariance_type.
    covariance_type : {'full', 'tied', 'diag', 'spherical'}
        The type of precision matrices.
    Returns
    -------
    precisions_cholesky : array-like
        The cholesky decomposition of sample precisions of the current
        components. The shape depends of the covariance_type.
    """
    estimate_precision_error_message = (
        "Fitting the mixture model failed because some components have "
        "ill-defined empirical covariance (for instance caused by singleton "
        "or collapsed samples). Try to decrease the number of components, "
        "or increase reg_covar.")

    if covariance_type == 'full':
        n_components, n_features, _ = covariances.shape
        precisions_chol = np.empty((n_components, n_features, n_features))
        for k, covariance in enumerate(covariances):
            try:
                cov_chol = linalg.cholesky(covariance, lower=True)
            except linalg.LinAlgError:
                raise ValueError(estimate_precision_error_message)
            precisions_chol[k] = linalg.solve_triangular(cov_chol,
                                                         np.eye(n_features),
                                                         lower=True).T
    elif covariance_type == 'tied':
        _, n_features = covariances.shape
        try:
            cov_chol = linalg.cholesky(covariances, lower=True)
        except linalg.LinAlgError:
            raise ValueError(estimate_precision_error_message)
        precisions_chol = linalg.solve_triangular(cov_chol, np.eye(n_features),
                                                  lower=True).T
    else:
        if np.any(np.less_equal(covariances, 0.0)):
            raise ValueError(estimate_precision_error_message)
        precisions_chol = 1. / np.sqrt(covariances)
    return precisions_chol
def _estimate_log_prob(means_,precisions_cholesky_,covariance_type,degrees_of_freedom_,mean_precision_, X):
        _, n_features = X.shape
        # We remove `n_features * np.log(degrees_of_freedom_)` because
        # the precision matrix is normalized
        log_gauss = (_estimate_log_gaussian_prob(
            X, means_, precisions_cholesky_, covariance_type) -
            .5 * n_features * np.log(degrees_of_freedom_))

        log_lambda = n_features * np.log(2.) + np.sum(digamma(
            .5 * (degrees_of_freedom_ -
                  np.arange(0, n_features)[:, np.newaxis])), 0)

        return log_gauss + .5 * (log_lambda -
                                 n_features / mean_precision_)
def _compute_log_det_cholesky(matrix_chol, covariance_type, n_features):
    """Compute the log-det of the cholesky decomposition of matrices.
    Parameters
    ----------
    matrix_chol : array-like
        Cholesky decompositions of the matrices.
        'full' : shape of (n_components, n_features, n_features)
        'tied' : shape of (n_features, n_features)
        'diag' : shape of (n_components, n_features)
        'spherical' : shape of (n_components,)
    covariance_type : {'full', 'tied', 'diag', 'spherical'}
    n_features : int
        Number of features.
    Returns
    -------
    log_det_precision_chol : array-like, shape (n_components,)
        The determinant of the precision matrix for each component.
    """
    if covariance_type == 'full':
        n_components, _, _ = matrix_chol.shape
        log_det_chol = (np.sum(np.log(
            matrix_chol.reshape(
                n_components, -1)[:, ::n_features + 1]), 1))

    elif covariance_type == 'tied':
        log_det_chol = (np.sum(np.log(np.diag(matrix_chol))))

    elif covariance_type == 'diag':
        log_det_chol = (np.sum(np.log(matrix_chol), axis=1))

    else:
        log_det_chol = n_features * (np.log(matrix_chol))

    return log_det_chol
def _estimate_gaussian_parameters(X, resp, reg_covar, covariance_type):
    """Estimate the Gaussian distribution parameters.
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        The input data array.
    resp : array-like, shape (n_samples, n_components)
        The responsibilities for each data sample in X.
    reg_covar : float
        The regularization added to the diagonal of the covariance matrices.
    covariance_type : {'full', 'tied', 'diag', 'spherical'}
        The type of precision matrices.
    Returns
    -------
    nk : array-like, shape (n_components,)
        The numbers of data samples in the current components.
    means : array-like, shape (n_components, n_features)
        The centers of the current components.
    covariances : array-like
        The covariance matrix of the current components.
        The shape depends of the covariance_type.
    """
    nk = resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps
    means = np.dot(resp.T, X) / nk[:, np.newaxis]
    covariances = {"full": _estimate_gaussian_covariances_full                   
                  }[covariance_type](resp, X, nk, means, reg_covar)
    return nk, means, covariances
def _estimate_gaussian_covariances_full(resp, X, nk, means, reg_covar):
    """Estimate the full covariance matrices.
    Parameters
    ----------
    resp : array-like, shape (n_samples, n_components)
    X : array-like, shape (n_samples, n_features)
    nk : array-like, shape (n_components,)
    means : array-like, shape (n_components, n_features)
    reg_covar : float
    Returns
    -------
    covariances : array, shape (n_components, n_features, n_features)
        The covariance matrix of the current components.
    """
    n_components, n_features = means.shape
    covariances = np.empty((n_components, n_features, n_features))
    for k in range(n_components):
        diff = X - means[k]
        covariances[k] = np.dot(resp[:, k] * diff.T, diff) / nk[k]
        covariances[k].flat[::n_features + 1] += reg_covar
    return covariances
def initialization(X,K,numiters,r0,alpha=1):
    ########################### Data preprocessing
    X,z,thetas,N,params,D=preprocess(X,K)

    clusters, sizes = np.unique(z, return_counts=True)
    m_para=sizes/N
    F=np.zeros(N)
#     params=tuple((np.array(pd.DataFrame(X).mean()),((np.array(pd.DataFrame(X).cov()))), 1, D))
    
#     m_para,F=F_est(np.ones,N,N,thetas,params,X)
    
    threshold=0.3
    
    I=(np.random.binomial(1, threshold, N))
        
    return X,z,I,thetas,N,params,D,clusters,sizes,m_para,F,threshold
def convergence_check(thetas,centroids_old,conv_criteria):
    centroids=np.copy(np.array(list([thetas[i][0] for i in range(len(thetas))])))
    if len(centroids)<len(centroids_old):
        change=np.sum(list(np.min(np.abs(np.linalg.norm(centroids_old-centroids[i],axis=1))) for i in range(len(centroids))))
    else:
        change=np.sum(list(np.min(np.abs(np.linalg.norm(centroids_old[i]-centroids,axis=1))) for i in range(len(centroids_old))))
    return change
def preprocess(X,K):
    
# Try different mean precision prior ie params[2] : No difference
# mean_precision_prior float | None, optional.
# The precision prior on the mean distribution (Gaussian). Controls the extent of where means can be placed. 
# Larger values concentrate the cluster means around mean_prior. The value of the parameter must be greater 
# than 0. If it is None, it is set to 1.

# Try different reg_covar : Too volatile
    if type(X) == list:
        X = np.array(X)
    if len(X.shape) == 1:
        X = X[:,np.newaxis]
    N = X.shape[0] #rows: observations
    D = X.shape[1] #columns: dimensions

    # Fit your data on the scaler object
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
#     X=normalize(X)
    # Initialize z
#     z=np.random.randint(K,size=N)
    z=KMeans(K).fit(X).predict(X)
    z+=1

    if D>1:
        params = tuple((np.mean(X,axis=0),(np.cov(X.T)), 1, D))
    elif D==1:
        params = tuple((np.mean(X),(np.var(X.T)),1, D))

    z=np.random.randint(K,size=N)

    thetas=[normalinvwishartsample(params) for k in range(K)]

    return X,z,thetas,N,params,D
def remove_cluster_new(X,z,K,thetas,params):
#     if len(thetas)>len(np.unique(np.abs(z))):
#         print(len(thetas)-len(np.unique(np.abs(z)))," clusters removed", len(np.unique(np.abs(z))) )
    N=len(z)
    z_temp=np.copy(z)
    clusters, sizes = np.unique(np.abs(z_temp), return_counts=True)

    c2=pd.DataFrame(clusters).copy()
    temp=c2.index.copy()+1
    c2.index=c2[0].copy()
    c2[0]=temp.copy()
    z=np.multiply(np.copy(c2[0][np.abs(z_temp)]),np.sign(z_temp+0.5))
    
    clusters, sizes = np.unique(np.abs(z), return_counts=True)
    K=len(clusters)
    
    return z,K,thetas
def compute_mixture_pdf(means_,precisions_cholesky_,covariance_type,mean_precision_, X,N,sizes):
    degrees_of_freedom_=sizes+X.shape[1]
    log_probs=_estimate_log_prob(means_,precisions_cholesky_,covariance_type,degrees_of_freedom_,mean_precision_, X)
    MN=(np.exp(log_probs))
    F=np.dot(sizes/N,MN.T)
    return degrees_of_freedom_,log_probs,MN,F
def compute_cluster_params(z,X,params,clusters,sizes,ind_matrix,reg_covar,covariance_type):
    K=(len(clusters))
    N=len(z)
    thetas=[]
    for k in (clusters-1):
        ind_k=np.where((z) == (k+1))[0]
        c = len(ind_k)
        if c<1:
#             print("Group anomaly")
            ind_k=np.where(np.abs(z) == (k+1))[0]
            c = len(ind_k)
        thetas.append((_estimate_gaussian_parameters(X[ind_k], 
                                            np.ones((c,1)), reg_covar, covariance_type)[1:3]))    
    nk=sizes
    means_=np.array([thetas[k][0].T for k in clusters-1])[:,:,0]
    covariances=np.array([thetas[k][1][0,:,:] for k in clusters-1])
    para_tuple=nk,means_,covariances
    precisions_cholesky_= np.array([_compute_precision_cholesky(cov, 
                                                covariance_type) for cov in [covariances]])[0,:,:,:]
    
    return para_tuple,thetas,nk,means_,covariances,precisions_cholesky_
def update_anomaly_labels(N,F,u,K,z,threshold,ppsa):
    prob=1-ppsa
    I=np.array(list(np.random.binomial(1,prob[i],1)[0] for i in range(N)))
    z=(np.abs(z)*np.power(-1,I))
    I[z<0]=1
    for k in range(K):
        ind_k=((z) == k)
        c=sum(ind_k)
        if (c<0.1*N) or ((c>0) and (np.mean(I[ind_k])>=0.75)):
            I[ind_k]=1
            z[ind_k]=np.abs(z[ind_k])*(-1)
    #     else:
    #         I[ind_k]=np.zeros(c)
    #         z[ind_k]=np.abs(z[ind_k])

    if np.mean(I)>0.3:
        I[np.argsort(F)[np.int(0.3*N):]]=0
        z[np.argsort(F)[np.int(0.3*N):]]=np.abs(z[np.argsort(F)[np.int(0.3*N):]])
        
#     if np.sum(z<0)<(0.01*N):
    if np.sum(z<0)==0:
        min_ana_count=max(np.int(0.01*N),10)
        I[np.argsort(F)[:min_ana_count]]=1
        z[np.argsort(F)[:min_ana_count]]=np.abs(z[np.argsort(F)[:min_ana_count]])*(-1)
#         print("min_ana_count",min_ana_count, "sum(I)", sum(I))

    threshold=min(max(beta.ppf(threshold , sum(I)+1,N-sum(I)+1),4/N,0.01), 0.3)

    return prob,I,threshold,z 
def _log_wishart_norm(degrees_of_freedom, log_det_precisions_chol, n_features):
    """Compute the log of the Wishart distribution normalization term.
    Parameters
    ----------
    degrees_of_freedom : array-like, shape (n_components,)
        The number of degrees of freedom on the covariance Wishart
        distributions.
    log_det_precision_chol : array-like, shape (n_components,)
         The determinant of the precision matrix for each component.
    n_features : int
        The number of features.
    Return
    ------
    log_wishart_norm : array-like, shape (n_components,)
        The log normalization of the Wishart distribution.
    """
    # To simplify the computation we have removed the np.log(np.pi) term
    return -(degrees_of_freedom * log_det_precisions_chol +
             degrees_of_freedom * n_features * .5 * math.log(2.) +
             np.sum(gammaln(.5 * (degrees_of_freedom -
                                  np.arange(n_features)[:, np.newaxis])), 0))
def compute_log_likelihhod(z,sizes,K,precisions_cholesky_,covariance_type,features,degrees_of_freedom_,
                           mean_precision_):
        # Contrary to the original formula, we have done some simplification
        # and removed all the constant terms.
        log_resp=np.log(np.abs(z))
        weight_concentration_ = (
                1. + sizes,
                (1/K +
                 np.hstack((np.cumsum(sizes[::-1])[-2::-1], 0))))

        # We removed `.5 * features * np.log(degrees_of_freedom_)`
        # because the precision matrix is normalized.
        log_det_precisions_chol = (_compute_log_det_cholesky(
            precisions_cholesky_, covariance_type, features) -
            .5 * features * np.log(degrees_of_freedom_))

        log_wishart = np.sum(_log_wishart_norm(
            degrees_of_freedom_, log_det_precisions_chol, features))
        
        log_norm_weight = -np.sum(betaln(weight_concentration_[0],
                                         weight_concentration_[1]))

        curr_log_likelihood=(-np.sum(np.exp(log_resp) * log_resp) -
                log_wishart - log_norm_weight -
                0.5 * features * np.sum(np.log(mean_precision_)))
        return curr_log_likelihood
def ppsa_vals(F,I,threshold):
    N=len(F)
    u=np.unique(F)
    ps1=F
    domain=((u>np.quantile(F,0.01))*1==(u<np.quantile(F,0.3))*1)
    G_Y_domain=u*domain
    G_Y_domain=(G_Y_domain[G_Y_domain>0])

    G_Y=domain*[np.sum(F[(F<=u)]) for n,u in enumerate(np.unique(F))]
    G_Y=G_Y[G_Y>0]
    G_Y=np.array(G_Y/max(G_Y))
    
    g_Y=np.diff(G_Y)/np.diff(G_Y_domain)

    aa=np.array(list(stats.percentileofscore(F, i)/100 for i in np.unique(F)))
    th3=aa[aa<0.3][np.argmax(np.abs(np.diff(np.quantile(G_Y,aa[aa<0.3]))))]
#     th3=threshold
    len_tail=np.int(th3*N) #drop in F

    u=np.quantile(ps1, th3)
    inds = np.where(ps1<=u)[0]

    iz=np.union1d(inds, np.where(I==1)[0])
#     inds0=iz[np.argsort(ps1[iz])[:min(np.int(0.3*N),len(iz),np.int(th3*N))]]
    inds0=np.argsort(ps1)[:min(np.int(0.3*N),len(iz),np.int(th3*N))]
    psa = np.abs(ps1[inds0] - u) 

    gpdparams = stats.genpareto.fit(psa)
    i_ppsa=np.zeros(N)
    
    i_ppsa[inds0] = stats.genpareto(1,0,gpdparams[2]).cdf(psa)
    ppsa=np.ones(N)
    ppsa[inds0]=1-(i_ppsa[inds0])
    
#     i_ppsa[iz] = stats.genpareto(1,0,gpdparams[2]).cdf(np.abs(ps1[iz] - u))
#     ppsa=np.ones(N)
#     ppsa[iz]=1-(i_ppsa[iz])
    
    ppsa[ppsa>1]=1
    ppsa[ppsa==0]=sys.float_info.min
    return ppsa,i_ppsa, inds, inds0,u,F, threshold

In [7]:
def incad_new_labels5(X,K,numiters,r0,conv_criteria):
# if 1:
    # initialization
    start_time=time.time()
    
    output={}

    log_likelihood=[]
    converged_=0
    covariance_type="full"
    
    X,z,I,thetas,N,params,features,clusters,sizes,m_para,F,threshold = initialization(X,K,numiters,r0,alpha=1)
    
    DistanceMatrix = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(X, 'euclidean'))
    
    mean_precision_=params[2]
    
    reg_covar=np.power(np.unique(DistanceMatrix)[1],1)/2

    niw_mat=np.array(list(normalinvwishartmarginal(X[i:i+1],
                         tuple((np.mean(X[i:i+1],axis=0),params[1],params[2],params[3]))
                                                  ) for i in range(N)))

    clusters,sizes=np.unique(np.abs(z).astype('int'),return_counts=True)
    
    ind_matrix=np.array(list(1*(np.abs(z).astype('int')==c) for c in clusters.astype('int'))) # all points  
    
    para_tuple,thetas,nk,means_,covariances,precisions_cholesky_= compute_cluster_params(z,X,
                                                    params,clusters,sizes,ind_matrix,reg_covar,covariance_type)
    
    degrees_of_freedom_,log_probs,MN,F = compute_mixture_pdf(means_,precisions_cholesky_,
                                         covariance_type,mean_precision_, X,N,sizes)
    
    if r0:
        ppsa,i_ppsa, inds, inds0,u,F, threshold = ppsa_vals(F,I,threshold)
        alpha2=1/(ppsa**r0)
        prob,I,threshold,z = update_anomaly_labels(N,F,u,K,z,threshold,ppsa)
        
    else:
        alpha2=np.ones(N)
        prob=np.zeros(N)
        ppsa=i_ppsa=inds=inds0=[]
        u=0
        threshold=0
        
    init_time=time.time()

    for n in range(numiters):
        sys.stdout.write('*'); sys.stdout.flush()
#         print(sizes)

        ps_new_clust=((alpha2/(N + alpha2 - 1))*niw_mat)[:,np.newaxis]

        ps_log=np.array([np.hstack(((np.log((sizes-(clusters==k0))[:,np.newaxis])-np.log(N+alpha2-1)).T+log_probs,
                    np.log(ps_new_clust))) for k0 in clusters])
        
        z=np.array(list(((1+np.argmax(np.random.multinomial(1, 
                (np.exp(ps_log[np.int(np.abs(z[i])-1),i,:])+sys.float_info.min)/np.sum(
                    np.exp(ps_log[np.int(np.abs(z[i])-1),i,:])+sys.float_info.min), 
                            size=1)))*np.power(-1,np.random.binomial(1,prob[i],1)[0])) for i in range(N)))
        
        # Update labels : drop empty clusters 
        z,K,thetas = remove_cluster_new(X,z,K,thetas,params)

        clusters,sizes=np.unique(np.abs(z).astype('int'),return_counts=True)
        
        ind_matrix=np.array(list(1*(np.abs(z).astype('int')==c) for c in clusters.astype('int'))) # all points  

        para_tuple,thetas,nk,means_,covariances,precisions_cholesky_= compute_cluster_params(z,X,
                                                        params,clusters,sizes,ind_matrix,reg_covar,covariance_type)

        degrees_of_freedom_,log_probs,MN,F = compute_mixture_pdf(means_,precisions_cholesky_,
                                             covariance_type,mean_precision_, X,N,sizes)

        if r0:
            ppsa,i_ppsa, inds, inds0,u,F, threshold = ppsa_vals(F,I,threshold)
            alpha2=1/(ppsa**r0)
            prob,I,threshold,z = update_anomaly_labels(N,F,u,K,z,threshold,ppsa)
            
        
        log_likelihood.append(compute_log_likelihhod(z,sizes,K,precisions_cholesky_,covariance_type,features,degrees_of_freedom_,
                           mean_precision_))
        
        if n > 50 and (np.max(np.abs(np.diff(log_likelihood[-3:])))<conv_criteria):
            converged_ += 1
            if converged_>0:
                print("Converged")
                break
                
    batch_time=time.time()
            
    output={}
    output['n']=n
    output['X']=np.copy(X)
    output['time_lapsed_init_ms']=(init_time-start_time)*1000.0
    output['time_lapsed_stream_ms']=(time.time()-batch_time)*1000.0
    output['time_lapsed_batch_ms']=(batch_time-init_time)*1000.0
    output['z']=np.copy(z)
    output['u']=np.copy(u)
    output['K']=copy.copy(K)
    output['r0']=copy.copy(r0)
    output['F']=np.copy(F)
    output['I']=np.copy(I)
    output['prob']=np.copy(prob)
    output['alpha2']=np.copy(alpha2)
    output['thetas']=thetas[:]
    output['log_likelihood']=copy.copy(log_likelihood)
    output['threshold']=copy.copy(threshold)
    output['ppsa']=np.copy(ppsa)
    output['i_ppsa']=np.copy(i_ppsa)
    output['inds']=np.copy(inds)
    output['inds0']=np.copy(inds0) 
    output['converged_']=converged_
    output['niw_mat']=np.copy(niw_mat)
    output['DistanceMatrix']=np.copy(DistanceMatrix)
    output['reg_covar']=np.copy(reg_covar)
    output['ind_matrix']=np.copy(ind_matrix)
    output['para_tuple']=copy.copy(para_tuple)
    output['means_']=np.copy(means_)
    output['covariances']=np.copy(covariances)
    output['precisions_cholesky_']=np.copy(precisions_cholesky_)
    output['degrees_of_freedom_']=np.copy(degrees_of_freedom_)
    output['log_probs']=np.copy(log_probs)
    output['MN']=np.copy(MN)
    output['params']=np.copy(params)
    
    return output           

In [8]:
def incad_new_labels_stream(X_full,K,batch_proportion,numiters,r0,conv_criteria):
# if 1:
    if 1:
        start_time=time.time()
        output={}

        log_likelihood=[]
        converged_=0
        covariance_type="full"

        batch_size=np.int(X_full.shape[0]*batch_proportion)

        N_full,D=X_full.shape

        N=copy.copy(batch_size)

        X=X_full[:batch_size]
        
        batch_output=incad_new_labels5(X,K,numiters,r0,conv_criteria)
        
        n=batch_output['n']
        X=batch_output['X']
        time_lapsed_init_ms=batch_output['time_lapsed_init_ms']
        time_lapsed_stream_ms=batch_output['time_lapsed_stream_ms']
        time_lapsed_batch_ms=batch_output['time_lapsed_batch_ms']
        z=batch_output['z']
        u=batch_output['u']
        K=batch_output['K']
        r0=batch_output['r0']
        F=batch_output['F']
        I=batch_output['I']
        prob=batch_output['prob']
        alpha2=batch_output['alpha2']
        thetas=batch_output['thetas']
        log_likelihood=batch_output['log_likelihood']
        threshold=batch_output['threshold']
        ppsa=batch_output['ppsa']
        i_ppsa=batch_output['i_ppsa']
        inds=batch_output['inds']
        inds0=batch_output['inds0'] 
        converged_=batch_output['converged_']
        niw_mat=batch_output['niw_mat']
        DistanceMatrix=batch_output['DistanceMatrix']
        reg_covar=batch_output['reg_covar']
        ind_matrix=batch_output['ind_matrix']
        para_tuple=batch_output['para_tuple']
        means_=batch_output['means_']
        covariances=batch_output['covariances']
        precisions_cholesky_=batch_output['precisions_cholesky_']
        degrees_of_freedom_=batch_output['degrees_of_freedom_']
        log_probs=batch_output['log_probs']
        MN=batch_output['MN']
        params=batch_output['params']


        mean_precision_=params[2]
        
        clusters,sizes=np.unique(np.abs(z).astype('int'),return_counts=True)

        batch_time=time.time()
    
    print("Begin Streaming")
    
    for i in np.arange(batch_size,N_full,1):
        sys.stdout.write('*'); sys.stdout.flush()
#         if i%100==0:
#             print(sizes)
        
        # Need full obs info        
        X=X_full[:i+1]
        
        if type(X) == list:
            X = np.array(X)
        if len(X.shape) == 1:
            X = X[:,np.newaxis]

        # Fit your data on the scaler object
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
        
        if i%(0.1*N)==0:
            if D>1:
                params = tuple((np.mean(X,axis=0),(np.cov(X.T)), 1, D))
            elif D==1:
                params = tuple((np.mean(X),(np.var(X.T)),1, D))
            DistanceMatrix = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(X[-np.int(0.1*N+1):,:],
                                                                                            'euclidean'))
            mean_precision_=params[2]
            reg_covar=np.min([reg_covar,np.power(np.unique(DistanceMatrix)[1],1)/2])

        z=np.append(z,K+1)
        I=np.append(I,0)
        prob=np.append(prob,0)
        N=N+1
        niw_mat=np.append(niw_mat,normalinvwishartmarginal(X[-1][:,np.newaxis],
                                        tuple((X[-1],params[1],params[2],params[3]))))

        clusters,sizes=np.unique(np.abs(z).astype('int'),return_counts=True)

        ind_matrix=np.array(list(1*(np.abs(z).astype('int')==c) for c in clusters.astype('int'))) # all points  

        para_tuple,thetas,nk,means_,covariances,precisions_cholesky_= compute_cluster_params(z,X,
                                                        params,clusters,sizes,ind_matrix,reg_covar,covariance_type)

        degrees_of_freedom_,log_probs,MN,F = compute_mixture_pdf(means_,precisions_cholesky_,
                                             covariance_type,mean_precision_, X,N,sizes)
        
        
        if r0:
            ppsa,i_ppsa, inds, inds0,u,F, threshold = ppsa_vals(F,I,threshold)
            alpha2=1/(ppsa**r0)
            prob,I,threshold,z = update_anomaly_labels(N,F,u,K,z,threshold,ppsa)
        
        
        # Update only tail points
        inds_update=np.union1d(inds0,np.where(I==1))
        inds_update=np.union1d(inds_update,i)
        

        ps_new_clust=((alpha2[inds_update]/(N + alpha2[inds_update] - 1))*niw_mat[inds_update])[:,np.newaxis]

        ps_log=np.array([np.hstack(((np.log((sizes-(clusters==k0))[:,np.newaxis])-np.log(N+alpha2[inds_update]-1)).T
                                    +log_probs[inds_update],
                    np.log(ps_new_clust))) for k0 in clusters])
        
        ps_log+=sys.float_info.min
        
        z[inds_update]=np.array(list(((1+np.argmax(np.random.multinomial(1, 
                (np.exp(ps_log[np.int(np.abs(z[i])-1),ii,:])+sys.float_info.min)/np.sum(
                    np.exp(ps_log[np.int(np.abs(z[i])-1),ii,:])+sys.float_info.min), 
                            size=1)))*np.power(-1,np.random.binomial(1,prob[i],1)[0])) 
                                      for ii,i in enumerate(inds_update)))
        
        # Update labels : drop empty clusters 
        z,K,thetas = remove_cluster_new(X,z,K,thetas,params)

    clusters,sizes=np.unique(np.abs(z).astype('int'),return_counts=True)

    ind_matrix=np.array(list(1*(np.abs(z).astype('int')==c) for c in clusters.astype('int'))) # all points  

    para_tuple,thetas,nk,means_,covariances,precisions_cholesky_= compute_cluster_params(z,X,
                                                    params,clusters,sizes,ind_matrix,reg_covar,covariance_type)

    degrees_of_freedom_,log_probs,MN,F = compute_mixture_pdf(means_,precisions_cholesky_,
                                         covariance_type,mean_precision_, X,N,sizes)

    output={}
    output['n']=n
    output['X']=np.copy(X)
    output['time_lapsed_init_ms']=time_lapsed_init_ms
    output['time_lapsed_stream_ms']=(time.time()-batch_time)*1000.0
    output['time_lapsed_batch_ms']=time_lapsed_batch_ms
    output['z']=np.copy(z)
    output['u']=np.copy(u)
    output['K']=copy.copy(K)
    output['r0']=copy.copy(r0)
    output['F']=np.copy(F)
    output['I']=np.copy(I)
    output['prob']=np.copy(prob)
    output['alpha2']=np.copy(alpha2)
    output['thetas']=thetas[:]
    output['log_likelihood']=copy.copy(log_likelihood)
    output['threshold']=copy.copy(threshold)
    output['ppsa']=np.copy(ppsa)
    output['i_ppsa']=np.copy(i_ppsa)
    output['inds']=np.copy(inds)
    output['inds0']=np.copy(inds0) 
    output['converged_']=converged_
    output['niw_mat']=np.copy(niw_mat)
    output['DistanceMatrix']=np.copy(DistanceMatrix)
    output['reg_covar']=np.copy(reg_covar)
    output['ind_matrix']=np.copy(ind_matrix)
    output['para_tuple']=copy.copy(para_tuple)
    output['means_']=np.copy(means_)
    output['covariances']=np.copy(covariances)
    output['precisions_cholesky_']=np.copy(precisions_cholesky_)
    output['degrees_of_freedom_']=np.copy(degrees_of_freedom_)
    output['log_probs']=np.copy(log_probs)
    output['MN']=np.copy(MN)
    output['params']=np.copy(params)
    
    return output        

# Code

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
import os
# date_label=datetime.now().strftime("_%m_%d_%Y")
date_label='_05_11_2020'
newpath = os.getcwd()+'/'+date_label
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
data_source_ad=os.getcwd()+'/Data/AD/'
data_source_clust=os.getcwd()+'/Data/Clustering/'
data_source_stream=os.getcwd()+'/Data/Stream/'
data_source_large=os.getcwd()+'/Data/Stream/Large'
if not os.path.exists(data_source_large):
    os.makedirs(data_source_large)

results_path_stream=newpath+'/Results/streaming/'
results_path_non_stream_ad=newpath+'/Results/non_streaming/AD/'
results_path_non_stream_clust=newpath+'/Results/non_streaming/Clustering/'

image_path_stream=results_path_stream+'images/'
image_path_non_stream_ad=results_path_non_stream_ad+'images/'
image_path_non_stream_clust=results_path_non_stream_clust+'images/'

if not os.path.exists(image_path_stream):
    os.makedirs(image_path_stream)
if not os.path.exists(image_path_non_stream_ad):
    os.makedirs(image_path_non_stream_ad)
if not os.path.exists(image_path_non_stream_clust):
    os.makedirs(image_path_non_stream_clust)

#### Model results

In [11]:
from scipy.io import arff
def load_data(file_path):
    filename, extension = os.path.splitext(file_path)
    name=(os.path.basename(file_path))
    if (extension=='.mat'):
        try:
            mat = scipy.io.loadmat(file_path)
            df = pd.DataFrame(np.hstack((mat['X'], mat['y'])))
            X,y=mat['X'], mat['y']
            return df,X,y

        except:
            try:
                arrays = {}
                f = h5py.File(file_path)
                for k, v in f.items():
                    arrays[k] = np.array(v)
                X,y=arrays['X'].T,arrays['y'].T
                df = pd.DataFrame(np.hstack((arrays['X'].T, arrays['y'].T)))
                return df,X,y
            except:
                print("1 Failed to load", name)

    elif extension=='.csv':
        try:
            df = pd.read_csv(file_path,low_memory=False,delimiter=',',header=None)
            df1=df[df.columns[(df.dtypes=='float')+(df.dtypes=='int')]]
            if df1.shape[1]==0:
                df = pd.read_csv(file_path,low_memory=False,delimiter=',')
                df1=df[df.columns[(df.dtypes=='float')+(df.dtypes=='int')]]                
            X=np.array(df1).astype(float)
            y=df.drop(df.columns[(df.dtypes=='float')+(df.dtypes=='int')],axis=1)
            return df,X,y
        except:
            try:
                df = pd.read_csv(file_path,low_memory=False,delimiter=',')
                df1=df[df.columns[(df.dtypes=='float')+(df.dtypes=='int')]]
                X=np.array(df1).astype(float)
                y=df.drop(df.columns[(df.dtypes=='float')+(df.dtypes=='int')],axis=1)
                return df,X,y
            except:
                print("2 Failed to load", name)

    elif extension=='.pickle':
        try:
            data=pickle.load(open(file_path,'rb'))
            X=data['X']
            y=data['y']
            return data,X,y
        except:
            try:
                d = pickle.load( open(file_path, "rb" ) )
                data=d['rawdata']
                labels=d['labels']
                X=np.array(data).astype(float)
                y=labels
                df=np.hstack((X,y))
                return df,X,y
            except:
                print("3 Failed to load", name)
                
    elif extension=='.arff':
        data = arff.loadarff(file_path)
        df = pd.DataFrame(data[0])
        df1=df[df.columns[(df.dtypes=='float')+(df.dtypes=='int')]]
        X=np.array(df1).astype(float)
        y=df.drop(df.columns[(df.dtypes=='float')+(df.dtypes=='int')],axis=1)
        return df,X,y
        
    else:
        print("Failed to load the extension",extension)
        pass

###### Non Stream- AD

In [18]:
# Anomaly Detection Non-Stream
onlyfiles = [f for f in listdir(data_source_ad) if isfile(join(data_source_ad, f))]
error_count=0
large_dataset_count=0
max_size=20001

K=10
r0=1
conv_criteria=0.1

for f in onlyfiles:
    file_path=data_source_ad+f
    try:
        df,X,y=load_data(file_path)
        XX=pd.DataFrame(X)
        X=XX[XX.columns[(XX.var()!=0)]]
        print(f,X.shape)
        numiters=np.max([np.int(len(X)/16),300])
        if X.shape[0]<max_size and X.shape[1]<=35:
            if X.shape[0]/X.shape[1]<5 or X.shape[1]>35:
                for n_comp in np.arange(25,X.shape[1],5):
                    pca = PCA(n_components=n_comp)
                    principalComponents = pca.fit_transform(X)
                    principalDf = pd.DataFrame(data = principalComponents)
                    if np.sum(pca.explained_variance_ratio_)>0.90 or n_comp==35:
                        break    
                X=np.array(principalDf)
            output=incad_new_labels5(X,K,numiters,r0,conv_criteria)
            output['df']=df
            output['y']=y
            pickle.dump( output, open(results_path_non_stream_ad+f+".pickle", "wb" ))

        else:
            large_dataset_count+=1
            shutil.move(file_path, data_source_large)
    except Exception as e:
        error_count+=1
        print(f)
        print(e)
print("large_dataset_count,error_count,len(onlyfiles)",large_dataset_count,error_count,len(onlyfiles))

http.mat (567498, 3)
http.mat
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version/Data/Stream/Large/http.mat' already exists
pima.mat (768, 8)
************************************************************************************************************************************************************************************************************************************************************************************************************wine.mat (129, 13)
************************************************************************************************************************************************************************************************************************************************************************************************************cardio.mat (1831, 21)
********************************************************************************************************************************************************************************

************************************************************************************************************************************************************************************************************************************************************************************************************letter-unsupervised-ad.csv (1600, 32)
************************************************************************************************************************************************************************************************************************************************************************************************************large_dataset_count,error_count,len(onlyfiles) 6 9 30


###### Non Stream- Clustering

In [19]:
# Clustering Non-Stream
onlyfolders = [f for f in listdir(data_source_clust) if not isfile(join(data_source_clust, f))]
for folder in onlyfolders:
    data_source_clust2=data_source_clust+folder+'/'
    onlyfiles=([f for f in listdir(data_source_clust2) if isfile(join(data_source_clust2, f))])

    error_count=0
    large_dataset_count=0
    max_size=20001

    K=25
    r0=1
    conv_criteria=0.1

    for f in onlyfiles:
        file_path=data_source_clust2+f
        try:
            df,X,y=load_data(file_path)
            XX=pd.DataFrame(X)
            X=XX[XX.columns[(XX.var()!=0)]]
            print(f,X.shape)
#             numiters=1
            numiters=np.max([np.int(len(X)/16),300])
            if X.shape[0]<max_size and X.shape[1]<=35:
                if X.shape[0]/X.shape[1]<5 or X.shape[1]>35:
                    for n_comp in np.arange(25,X.shape[1],5):
                        pca = PCA(n_components=n_comp)
                        principalComponents = pca.fit_transform(X)
                        principalDf = pd.DataFrame(data = principalComponents)
                        if np.sum(pca.explained_variance_ratio_)>0.90 or n_comp==35:
                            break    
                    X=np.array(principalDf)
                output=incad_new_labels5(X,K,numiters,r0,conv_criteria)
                output['df']=df
                output['y']=y
                pickle.dump( output, open(results_path_non_stream_clust+f+".pickle", "wb" ))

            else:
                large_dataset_count+=1
                shutil.move(file_path, data_source_large)

        except Exception as e:
            error_count+=1
            print(e)
    print("large_dataset_count,error_count,len(onlyfiles)",large_dataset_count,error_count,len(onlyfiles))

disk-4000n.arff (4000, 2)
************************************************************************************************************************************************************************************************************************************************************************************************************disk-5000n.arff (5000, 2)
************************************************************************************************************************************************************************************************************************************************************************************************************************zelnik4.arff (622, 2)
************************************************************************************************************************************************************************************************************************************************************************************************************chainlink.arff

************************************************************************************************************************************************************************************************************************************************************************************************************flame.arff (240, 2)
************************************************************************************************************************************************************************************************************************************************************************************************************simplex.arff (500, 3)
************************************************************************************************************************************************************************************************************************************************************************************************************smile3.arff (1000, 2)
************************************

************************************************************************************************************************************************************************************************************************************************************************************************************disk-1000n.arff (1000, 2)
************************************************************************************************************************************************************************************************************************************************************************************************************engytime.arff (4096, 2)
************************************************************************************************************************************************************************************************************************************************************************************************************dense-disk-3000.arff (3000, 2)
*******************

************************************************************************************************************************************************************************************************************************************************************************************************************pageb.preproc.csv (5473, 11)
******************************************************************************************************************************************************************************************************************************************************************************************************************************************************spambase.preproc.csv (4601, 58)
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version/Data/Stream/Large/spambase.preproc.csv' already exists
synthetic.preproc.csv (20000, 10)
********************************************************************************************************************

###### Stream

In [32]:
# AD Stream
onlyfolders = [f for f in listdir(data_source_stream) if (not isfile(join(data_source_stream, f)) and ('gas_sensors' not in f))]
for folder in onlyfolders:
    data_source_stream2=data_source_stream+folder+'/'
    onlyfiles=([f for f in listdir(data_source_stream2) if isfile(join(data_source_stream2, f))])
    error_count=0
    large_dataset_count=0

    K=10
    r0=1
    conv_criteria=0.1
    batch_proportion=0.35
    max_size=5000


    for f in onlyfiles:
        file_path=data_source_stream2+f
        try:
            df,X,y=load_data(file_path)
            if 'timestamp' in df.columns:
                df1=df
                df1['timestamp']=pd.to_datetime(df1['timestamp'])
                df1['new_date']=[d.date() for d in df1['timestamp']]
                df1['new_time']=[d.time() for d in df1['timestamp']]
                df1['mins']=[d.hour*60+d.minute for d in df1['new_time']]
                df1=df.drop(['timestamp','new_date','new_time'],axis=1)
                X=np.array(df1).astype(float)                
            XX=pd.DataFrame(X)
            X=XX[XX.columns[(XX.var()!=0)]]
            print(f,X.shape)
            numiters=1
            numiters=np.max([np.int(len(X)/16),300])
            if X.shape[0]<max_size:
                if X.shape[0]/X.shape[1]<5 or X.shape[1]>35:
                    for n_comp in np.arange(25,X.shape[1],5):
                        pca = PCA(n_components=n_comp)
                        principalComponents = pca.fit_transform(X)
                        principalDf = pd.DataFrame(data = principalComponents)
                        if np.sum(pca.explained_variance_ratio_)>0.90 or n_comp==35:
                            break    
                    X=np.array(principalDf)
                output=incad_new_labels_stream(X,K,batch_proportion,numiters,r0,conv_criteria)
                output['df']=df
                output['y']=y
                pickle.dump( output, open(results_path_stream+f+".pickle", "wb" ))
            else:
                large_dataset_count+=1
                shutil.move(file_path, data_source_large)
        except Exception as e:
            error_count+=1
            print(e)
    print("large_dataset_count,error_count,len(onlyfiles)",large_dataset_count,error_count,len(onlyfiles))
    print("")

ambient_temperature_system_failure.csv (7267, 2)
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version/Data/Stream/Large/ambient_temperature_system_failure.csv' already exists
http.mat (567498, 3)
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version/Data/Stream/Large/http.mat' already exists
machine_temperature_system_failure.csv (22695, 2)
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version/Data/Stream/Large/machine_temperature_system_failure.csv' already exists
skin.preproc.csv (245057, 4)
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version/Data/Stream/Large/skin.preproc.csv' already exists
mnist.mat (7603, 78)
Destination path '/Users/lekhag/Documents/INCAD Improvements/Parallel Gibbs/Cleaned Version/Data/Stream/Large/mnist.mat' already exists
Failed to load the extension 
cannot unpack non-iterable NoneType object
shuttle.prepr

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

###### GAS Sensor Data Results

In [329]:
mypath=data_source_stream+'gas_sensors/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['batch8.dat',
 'batch9.dat',
 'batch4.dat',
 'batch5.dat',
 'batch7.dat',
 'batch6.dat',
 'batch2.dat',
 'batch3.dat',
 'batch1.dat',
 'batch10.dat']

In [409]:
import plotly.express as px
for file_num,file in enumerate(onlyfiles):
    df0=pd.concat([pd.read_table(mypath+file, sep=' ',header=None, dtype='str')] )
    df0=df0.sort_values(by=0)
    print(file)
    for c in df0.columns:
        if c>0:
            df0[c]=df0[c].str.split(':').str[1]
    df0=df0.astype(float)

    df=df0.drop([0], axis=1)
    truth=np.array(df0[0])
    pca_df = PCA(n_components=10)
    X = pca_df.fit_transform(df)
    # print(np.sum(pca_df.explained_variance_ratio_),X.shape, df0.shape, df.shape, truth.shape)

    K=10
    numiters=100
    batch_proportion=0.25
    r0=1
    conv_criteria=0.001
    # output=incad_new_labels5(X,K,numiters,r0,conv_criteria)
    output=incad_new_labels_stream(X,K,batch_proportion,numiters,r0,conv_criteria)
    output['X']=X
    output['y']=truth
    name='Gas_sensor_'+str(file)
    pickle.dump( output, open( results_path_stream+name+".pickle", "wb" ) )
    print("saved")

batch8.dat
****************************************************************************************************Begin Streaming
*****************************************************************************************************************************************************************************************************************************saved
batch9.dat
****************************************************************************************************Begin Streaming
*****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************saved
batch4.dat
****************************************************************************************************Begin Streaming
*********************************

###### Large Non-Streaming Data Results

In [12]:
mypath=data_source_stream
onlyfolders = [f for f in listdir(mypath) if  not isfile(join(mypath, f)) and(f!="gas_sensors")]

onlyfiles=np.array([])
for folder in onlyfolders:
    ss=np.array([folder+"/"+f for f in listdir(mypath+folder+'/') if isfile(join(mypath+folder+'/', f))])
    onlyfiles=np.append(onlyfiles,ss)

In [13]:
for folder in onlyfolders:
    onlyfiles= [folder+"/"+f for f in listdir(mypath+folder+'/') if isfile(join(mypath+folder+'/', f))]
    for filename in onlyfiles:
        print(filename)
        
        try:
            df,X,y=load_data(mypath+filename)
            if ('timestamp' in df.columns) and (folder!='Stream_data'):
                shutil.move(mypath+filename, data_source_stream+'Stream_data')
            elif ('timestamp' not in df.columns) and (folder=='Stream_data'):
                shutil.move(mypath+filename, data_source_large)
        except Exception as e:
            print(e)

Large/http.mat
Large/skin.preproc.csv
Large/mnist.mat
Large/.DS_Store
Failed to load the extension 
cannot unpack non-iterable NoneType object
Large/arrhythmia.mat
Large/shuttle.preproc.csv
Large/shuttle.mat
Large/musk.mat
Large/shuttle-unsupervised-ad.csv
Large/birch-rg1.arff
Large/mnist.raw.pickle
'numpy.ndarray' object has no attribute 'columns'
Large/gas.preproc.csv
Large/optdigits.mat
Large/comm.and.crime.preproc.csv
Large/satellite.mat
Large/speech.mat
Large/satimage.pickle
'dict' object has no attribute 'columns'
Large/birch-rg3.arff
Large/sonar.arff
Large/birch-rg2.arff
Large/smtp.mat
Large/spambase.preproc.csv
Large/satimage-2.mat
Large/cover.mat
Large/kdd99-unsupervised-ad.csv
Large/arrhythmia.arff
Large/opt.digits.preproc.csv
Stream_data/ambient_temperature_system_failure.csv
Stream_data/machine_temperature_system_failure.csv
Stream_data/ec2_request_latency_system_failure.csv
Stream_data/.DS_Store
Failed to load the extension 
cannot unpack non-iterable NoneType object
Strea

In [23]:
mypath=data_source_large+'/'
onlyfiles= [f for f in listdir(mypath) if isfile(join(mypath, f)) and ('.DS_Store' not in f) ]


error_count=0
large_dataset_count=0
max_size=40001

K=10
r0=1
conv_criteria=0.1

for filename in onlyfiles[19:]:
    try:
        df,X,y=load_data(mypath+filename)
        XX=pd.DataFrame(X)
        X=XX[XX.columns[(XX.var()!=0)]]
        numiters=500
#         numiters=np.max([np.int(len(X)/16),500])
        if X.shape[0]<max_size:
            if X.shape[0]/X.shape[1]<5 or X.shape[1]>35:
                for n_comp in np.arange(25,X.shape[1],5):
                    pca = PCA(n_components=n_comp)
                    principalComponents = pca.fit_transform(X)
                    principalDf = pd.DataFrame(data = principalComponents)
                    if np.sum(pca.explained_variance_ratio_)>0.90 or n_comp==35:
                        break    
                X=np.array(principalDf)
            print(filename, X.shape)
            output=incad_new_labels5(X,K,numiters,r0,conv_criteria)
            output['df']=df
            output['y']=y
            pickle.dump( output, open(results_path_non_stream_ad+"LARGE_"+filename+".pickle", "wb" ))
        else:
            print(filename, X.shape)
            large_dataset_count+=1
    except Exception as e:
        error_count+=1
        print(filename)
        print(e)

print("large_dataset_count,error_count,len(onlyfiles)",large_dataset_count,error_count,len(onlyfiles))

smtp.mat (95156, 3)
spambase.preproc.csv (4601, 35)
***spambase.preproc.csv
Fitting the mixture model failed because some components have ill-defined empirical covariance (for instance caused by singleton or collapsed samples). Try to decrease the number of components, or increase reg_covar.
satimage-2.mat (5803, 25)
********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************cover.mat (286048, 10)
kdd99-unsupervised-ad.csv (620098, 29)
arrhythmia.arff (452, 30)
****************************************************Converged
opt.digits.preproc.csv (5620, 3

In [ ]:
mypath=data_source_large+'/'
onlyfiles= [f for f in listdir(mypath) if isfile(join(mypath, f)) and ('.DS_Store' not in f) ]


error_count=0
large_dataset_count=0
max_size=100001

K=10
r0=1
conv_criteria=0.1

for filename in onlyfiles:
    try:
        df,X,y=load_data(mypath+filename)
        XX=pd.DataFrame(X)
        X=XX[XX.columns[(XX.var()!=0)]]
        numiters=300
#         numiters=np.max([np.int(len(X)/16),500])
        if X.shape[0]<max_size and X.shape[0]>40000:
            if X.shape[0]/X.shape[1]<5 or X.shape[1]>35:
                for n_comp in np.arange(25,X.shape[1],5):
                    pca = PCA(n_components=n_comp)
                    principalComponents = pca.fit_transform(X)
                    principalDf = pd.DataFrame(data = principalComponents)
                    if np.sum(pca.explained_variance_ratio_)>0.90 or n_comp==35:
                        break    
                X=np.array(principalDf)
            print(filename, X.shape)
            output=incad_new_labels5(X,K,numiters,r0,conv_criteria)
            output['df']=df
            output['y']=y
            pickle.dump( output, open(results_path_non_stream_ad+"LARGE_"+filename+".pickle", "wb" ))
        else:
            print(filename, X.shape)
            large_dataset_count+=1
    except Exception as e:
        error_count+=1
        print(filename)
        print(e)

print("large_dataset_count,error_count,len(onlyfiles)",large_dataset_count,error_count,len(onlyfiles))

http.mat (567498, 3)
skin.preproc.csv (245057, 4)
mnist.mat (7603, 78)
arrhythmia.mat (452, 257)
shuttle.preproc.csv (58000, 10)


###### Streaming Data Results